<a href="https://colab.research.google.com/github/henryonomakpo/The-Impact-of-ESG-Ratings-on-EV-Manufacturing-Industry/blob/main/EV_ESG_PANEL_DATA_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### Fetch ESG data using yesg for EV Automakers

In [43]:
!pip install yfinance
!pip install yesg
!pip install statsmodels
!pip install xlsxwriter
!pip install linearmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.2 MB/s eta 0:00:00


### Fama-MacBeth  

In [73]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.style as style
import yfinance as yf
import statsmodels.api as sm
import numpy as np
import xlsxwriter
import warnings
from io import StringIO
import statsmodels.formula.api as smf # For formula API if preferred

# --- Standard Setup ---
warnings.filterwarnings("ignore", category=FutureWarning)
style.use("default")
params = {"axes.labelsize": 8, "font.size": 8, "legend.fontsize": 8, "xtick.labelsize": 8,
          "ytick.labelsize": 8, "text.usetex": False, "font.family": "sans-serif",
          "axes.spines.top": False, "axes.spines.right": False, "grid.color": "grey",
          "axes.grid": True, "grid.alpha": 0.5, "grid.linestyle": ":"}
plt.rcParams.update(params)

# --- Constants ---
ev_tickers_returns = ["TSLA", "NIO", "RIVN", "LCID", "XPEV",
                      "LI", "GM", "F", "005380.KS", "PSNY", "BMW.DE", "MBG.DE"]
ev_tickers_esg_components = ["TSLA", "NIO", "RIVN", "LCID", "XPEV",
                             "LI", "GM", "F", "005380.KS", "BMW.DE", "MBG.DE", "PSNY"]
csv_file_path = "/content/gd_Developed_5_Factors.csv" # MAKE SURE THIS IS CORRECT

# --- Data Downloading ---
adj_close_data = pd.DataFrame()
print("Downloading historical stock data (2019-2024)...")
download_start_date = "2019-11-01" # Start earlier for lagging
download_end_date = "2024-12-31" # Match factor data end
for ticker in ev_tickers_returns:
    try:
        data = yf.download(ticker, start=download_start_date, end=download_end_date, progress=False)
        if data.empty: print(f"No data for {ticker}. Skipping.") ; continue
        adj_close_data[ticker] = data.get("Adj Close", data["Close"])
    except Exception as e: print(f"Error downloading {ticker}: {e}")
print("Stock data download complete.")

# --- Calculate Monthly Returns ---
print("\nCalculating monthly returns...")
if adj_close_data.empty: raise ValueError("No stock data downloaded.")
monthly_returns = adj_close_data.resample("ME").last().pct_change() * 100
print("Monthly returns calculated.")

# --- Load Fama-French 6-Factor and ESG data ---
print(f"\nLoading Fama-French 6-Factor and ESG data from '{csv_file_path}'...")
try:
    _ff_esg_data_raw = pd.read_csv(
        csv_file_path, index_col=0, header=0, on_bad_lines='warn',
        parse_dates=[0], date_format='%m/%d/%y' # Verify this format matches your CSV!
    )
    print("Data loaded successfully.")
except FileNotFoundError: raise FileNotFoundError(f"CSV file not found: {csv_file_path}")
except ValueError as date_err: raise ValueError(f"Error parsing dates in CSV: {date_err}")
except Exception as e: raise Exception(f"Error loading CSV file: {e}")

# Define expected factor columns
factor_columns = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF', 'MOM']
esg_suffixes = ['_E', '_S', '_G', '_ESG']

# Build list of all expected columns
expected_columns = factor_columns[:]
for ticker in ev_tickers_esg_components:
    for suffix in esg_suffixes: expected_columns.append(f"{ticker}{suffix}")

# Filter the loaded data
actual_columns_in_csv = [col for col in expected_columns if col in _ff_esg_data_raw.columns]
essential_factors = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'MOM', 'RF']
if not all(f in actual_columns_in_csv for f in essential_factors):
     missing_factors = [f for f in essential_factors if f not in actual_columns_in_csv]
     raise ValueError(f"Core factors missing: {missing_factors}")
ff_esg_data = _ff_esg_data_raw[actual_columns_in_csv].copy()
print(f"Using {len(ff_esg_data.columns)} columns from CSV.")

# Convert factor data index to datetime PeriodIndex (MonthEnd)
try:
    if not isinstance(ff_esg_data.index, pd.DatetimeIndex): raise TypeError("Index not DatetimeIndex.")
    ff_esg_data.index = ff_esg_data.index.to_period('M').to_timestamp('M')
    ff_esg_data.dropna(axis=0, how='all', subset=factor_columns[:-1], inplace=True)
except Exception as e: raise ValueError(f"Error processing factor data index: {e}")


# --- Impute Missing ESG Data (Fill NaN with 0 - As per request) ---
print("\nImputing missing ESG data (Aggregate & Components) by filling NaN with 0...")
imputation_count = 0
for suffix in ['_ESG', '_E', '_S', '_G']:
    for ticker in ev_tickers_esg_components:
        col_name = f"{ticker}{suffix}"
        if col_name in ff_esg_data.columns:
            series = ff_esg_data[col_name]
            # Uncomment next line IF original zeros should be treated as missing BEFORE filling NaNs with 0
            # series = series.replace(0, np.nan)
            num_nan = series.isnull().sum()
            if num_nan > 0:
                ff_esg_data[col_name] = series.fillna(0)
                imputation_count += num_nan
print(f"ESG imputation complete. Filled {imputation_count} NaN values with 0.")

# --- Data Preparation for Fama-MacBeth ---

# 1. Align Returns and Factors/ESG
print("\nAligning returns and factor/ESG data by date...")
monthly_returns.index = monthly_returns.index.to_period('M').to_timestamp('M')
common_index_dates = monthly_returns.index.intersection(ff_esg_data.index)
if common_index_dates.empty: raise ValueError("No overlapping dates.")
start_date, end_date = common_index_dates.min(), common_index_dates.max()
print(f"Common date range: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
returns_aligned = monthly_returns.loc[common_index_dates]
factors_esg_aligned = ff_esg_data.loc[common_index_dates] # Use imputed data

# 2. Calculate Excess Returns (Wide Format)
print("Calculating excess returns...")
excess_returns_wide = pd.DataFrame(index=returns_aligned.index)
rf_series = factors_esg_aligned["RF"]
for ticker in ev_tickers_returns:
    if ticker in returns_aligned.columns:
        excess_returns_wide[ticker] = returns_aligned[ticker] - rf_series
print("Excess returns calculated.")

# 3. Prepare Factor Data (Common across all stocks)
factors_df = factors_esg_aligned[essential_factors[:-1]].copy() # FF6 factors: Mkt-RF to MOM

# 4. Prepare Lagged ESG Component Data (Stock-Specific) - LONG FORMAT for easier merging
print("Preparing lagged ESG component data...")
esg_components_long_list = []
component_suffixes = ['_E', '_S', '_G']
for ticker in ev_tickers_esg_components:
    temp_ticker_esg = pd.DataFrame({'Ticker': ticker}, index=factors_esg_aligned.index)
    processed_any = False
    for suffix in component_suffixes:
        col_name = f"{ticker}{suffix}"
        lagged_col_name = suffix[1:] + "_lagged" # e.g., E_lagged
        if col_name in factors_esg_aligned.columns:
            temp_ticker_esg[lagged_col_name] = factors_esg_aligned[col_name].shift(1)
            processed_any = True
        else: temp_ticker_esg[lagged_col_name] = np.nan
    if processed_any: esg_components_long_list.append(temp_ticker_esg)
if not esg_components_long_list: raise ValueError("Could not process ESG components.")
# *** CORRECTED VARIABLE NAME HERE ***
esg_components_with_index = pd.concat(esg_components_long_list) # Concatenate list
esg_lagged_long = esg_components_with_index.reset_index().rename(columns={'Date': 'Date'}) # Reset index

# 5. Combine Data & Handle NaNs (Resulting from Lagging)
print("Combining data and handling initial NaNs from lagging...")
# Convert returns to long format for merging
panel_data = excess_returns_wide.stack().reset_index()
panel_data.columns = ['Date', 'Ticker', 'excess_return']
# Merge factors (common by Date)
panel_data = pd.merge(panel_data, factors_df.reset_index(), on='Date', how='left')
# Merge lagged ESG (firm-specific by Date & Ticker)
panel_data = pd.merge(panel_data, esg_lagged_long, on=['Date', 'Ticker'], how='left')

# Drop rows where lag resulted in NaN for ESG OR if factors/returns were missing
essential_cols = ['excess_return', 'E_lagged', 'S_lagged', 'G_lagged'] + factors_df.columns.tolist()
initial_rows = len(panel_data)
panel_data.dropna(subset=essential_cols, inplace=True)
rows_after_na = len(panel_data)
print(f"Dropped {initial_rows - rows_after_na} rows due to missing values (incl. lag).")
if panel_data.empty: raise ValueError("No data remaining after handling lag NaNs.")

# --- Fama-MacBeth Regression Implementation ---
print("\n--- Running Fama-MacBeth Regression (FF6 + Lagged E,S,G) ---")

results_list = []
# Group final panel data by date for cross-sectional regressions
panel_data['Date'] = pd.to_datetime(panel_data['Date']) # Ensure Date is datetime object
grouped_by_date = panel_data.groupby('Date')
dates_run = [] # Keep track of dates where regression runs

for date, group in grouped_by_date:
    # --- Stage 1: Cross-Sectional Regressions for each date ---
    y_t = group['excess_return']
    # Select predictors FOR THIS DATE'S GROUP: Factors + Lagged E, S, G
    X_t_cols = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'MOM', 'E_lagged', 'S_lagged', 'G_lagged']
    # Check if all needed predictor columns are present in this group's data
    if not all (c in group.columns for c in X_t_cols):
        print(f"Warning: Skipping date {date.date()} due to missing predictor columns in group.")
        continue
    X_t = group[X_t_cols]
    X_t = sm.add_constant(X_t, has_constant='add') # Add constant for cross-section reg

    # Run OLS if enough firms (> #predictors + 1) remain for this date
    if len(y_t) > X_t.shape[1]:
        try:
            result_t = sm.OLS(y_t, X_t).fit()
            results_list.append(result_t.params) # Store coefficients
            dates_run.append(date) # Record the date
        except Exception as e_ols:
            print(f"Warning: OLS failed for date {date.date()}: {e_ols}")
    # else: # Optional: Print if too few firms
        # print(f"Skipping date {date.date()} due to insufficient firms ({len(y_t)} <= {X_t.shape[1]} predictors).")


# --- Stage 2: Analyze Time Series of Coefficients ---
if not results_list:
    print("\nERROR: No cross-sectional regressions could be run successfully.")
    fmb_summary_to_save = "Error: No cross-sectional regressions ran."
else:
    coeffs_df = pd.DataFrame(results_list, index=pd.to_datetime(dates_run))
    print(f"\nTime Series of Estimated Coefficients (Gammas) based on {len(coeffs_df)} dates:\n", coeffs_df.head())

    fmb_estimates = coeffs_df.mean()
    print("\nCalculating Fama-MacBeth t-statistics (using Newey-West SEs)...")
    t_stats = {}
    p_values = {}
    # Heuristic for Newey-West lags based on sample size T
    T = len(coeffs_df)
    nw_lags = min(4, T // 4) if T >= 16 else max(1, T // 3)
    print(f"Using Newey-West lag: {nw_lags}")

    for factor in coeffs_df.columns:
        coef_series = coeffs_df[factor].dropna()
        if coef_series.empty or coef_series.var() == 0 or len(coef_series) < 2:
             print(f"Warning: Cannot calculate NW t-stat for {factor} (empty, no variance, or < 2 obs).")
             t_stats[factor] = np.nan; p_values[factor] = np.nan; continue
        try:
            # Regress coefficient series on a constant using OLS with HAC errors
            est = sm.OLS(coef_series, sm.add_constant(np.ones(len(coef_series)))).fit(
                cov_type='HAC', cov_kwds={'maxlags': nw_lags}
                )
            if 'const' in est.params.index:
                t_stats[factor] = est.tvalues['const']
                p_values[factor] = est.pvalues['const']
            else:
                 print(f"Warning: Constant term not estimated for {factor} NW regression.")
                 t_stats[factor] = np.nan; p_values[factor] = np.nan
        except Exception as e_nw:
             print(f"Error calculating Newey-West SE for {factor}: {e_nw}")
             t_stats[factor] = np.nan; p_values[factor] = np.nan

    # Assemble final FMB results table
    fmb_results = pd.DataFrame({
        'Average_Premium': fmb_estimates,
        'T_statistic_NW': pd.Series(t_stats),
        'P_value_NW': pd.Series(p_values)
    })

    print("\n--- Fama-MacBeth Regression Results ---")
    # Format for better readability
    print(fmb_results.round(4))
    fmb_summary_to_save = fmb_results

# --- Save Fama-MacBeth Results to Excel ---
output_filename = 'fama_macbeth_results_ff6_ESG_components_impute0.xlsx' # Updated name
print(f"\nSaving Fama-MacBeth results to '{output_filename}'...")
try:
    with pd.ExcelWriter(output_filename, engine='xlsxwriter') as writer:
        if isinstance(fmb_summary_to_save, pd.DataFrame):
            fmb_summary_to_save.reset_index().rename(columns={'index':'Factor'}).to_excel(writer, sheet_name='Fama_MacBeth_Results', index=False)
            if 'coeffs_df' in locals() and not coeffs_df.empty:
                 coeffs_df.reset_index().rename(columns={'index':'Date'}).to_excel(writer, sheet_name='Coefficient_TimeSeries', index=False)
            print(f"Fama-MacBeth results saved successfully.")
        else: # Save error message if FMB failed
             pd.DataFrame({'Error': [fmb_summary_to_save]}).to_excel(writer, sheet_name='Error', index=False)
             print("Fama-MacBeth estimation did not complete, error message saved.")
except Exception as save_err:
    print(f"ERROR: Could not write Fama-MacBeth results to Excel: {save_err}")

Stock data download complete.

Calculating monthly returns...
Monthly returns calculated.

Loading Fama-French 6-Factor and ESG data from '/content/gd_Developed_5_Factors.csv'...
Data loaded successfully.
Using 55 columns from CSV.

Imputing missing ESG data (Aggregate & Components) by filling NaN with 0...
ESG imputation complete. Filled 689 NaN values with 0.

Aligning returns and factor/ESG data by date...
Common date range: 2019-12-31 to 2024-12-31
Calculating excess returns...
Excess returns calculated.
Preparing lagged ESG component data...
Combining data and handling initial NaNs from lagging...
Dropped 7 rows due to missing values (incl. lag).

--- Running Fama-MacBeth Regression (FF6 + Lagged E,S,G) ---

Time Series of Estimated Coefficients (Gammas) based on 43 dates:
                const    Mkt-RF       SMB       HML       RMW       CMA  \
2021-06-30  0.785838  0.793696 -0.793696 -4.109932  0.550087 -1.791710   
2021-07-31 -0.226422 -0.235479  0.489071  0.378124 -0.950972  

### Fetch ESG Data using yesg

In [69]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.style as style
import yfinance as yf
import statsmodels.api as sm
import numpy as np
import xlsxwriter
import warnings
from io import StringIO

# Import PanelOLS and RandomEffects from linearmodels
from linearmodels.panel import PanelOLS, RandomEffects

# --- Standard Setup (Warnings, Style) ---
warnings.filterwarnings("ignore", category=FutureWarning) # Suppress various FutureWarnings
style.use("default")
params = {"axes.labelsize": 8, "font.size": 8, "legend.fontsize": 8, "xtick.labelsize": 8,
          "ytick.labelsize": 8, "text.usetex": False, "font.family": "sans-serif",
          "axes.spines.top": False, "axes.spines.right": False, "grid.color": "grey",
          "axes.grid": True, "grid.alpha": 0.5, "grid.linestyle": ":"}
plt.rcParams.update(params)

# --- Constants ---
ev_tickers_returns = ["TSLA", "NIO", "RIVN", "LCID", "XPEV",
                      "LI", "GM", "F", "005380.KS", "PSNY", "BMW.DE", "MBG.DE"] # Updated list based on new dataset
ev_tickers_esg_components = ["TSLA", "NIO", "RIVN", "LCID", "XPEV",
                             "LI", "GM", "F", "005380.KS", "BMW.DE", "MBG.DE", "PSNY"] # Tickers expected to have E,S,G cols
csv_file_path = "/content/gd_Developed_5_Factors.csv" # Ensure this is correct

# --- Data Downloading (Returns) ---
adj_close_data = pd.DataFrame()
print("Downloading historical stock data (2019-2024)...")
download_start_date = "2019-11-01" # Start earlier for lagging
download_end_date = "2024-12-31" # Match factor data end
for ticker in ev_tickers_returns:
    try:
        data = yf.download(ticker, start=download_start_date, end=download_end_date, progress=False)
        if data.empty: print(f"No data for {ticker}. Skipping.") ; continue
        adj_close_data[ticker] = data.get("Adj Close", data["Close"])
        # print(f"Data downloaded for {ticker}.") # Less verbose
    except Exception as e: print(f"Error downloading {ticker}: {e}")
print("Stock data download complete.")

# --- Calculate Monthly Returns ---
print("\nCalculating monthly returns...")
if adj_close_data.empty: raise ValueError("No stock data downloaded.")
monthly_returns = adj_close_data.resample("ME").last().pct_change() * 100
print("Monthly returns calculated.")

# --- Load Fama-French 6-Factor and ESG data ---
print(f"\nLoading Fama-French 6-Factor and ESG data from '{csv_file_path}'...")
try:
    _ff_esg_data_raw = pd.read_csv(
        csv_file_path, index_col=0, header=0, on_bad_lines='warn',
        parse_dates=[0], date_format='%m/%d/%y' # Specify date format
    )
    print("Data loaded successfully.")
except FileNotFoundError: raise FileNotFoundError(f"CSV file not found: {csv_file_path}")
except ValueError as date_err: raise ValueError(f"Error parsing dates in CSV: {date_err}")
except Exception as e: raise Exception(f"Error loading CSV file: {e}")

# Define expected factor columns (FF5 + MOM + RF)
factor_columns = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF', 'MOM']
esg_suffixes = ['_E', '_S', '_G', '_ESG'] # Include aggregate score column

# Build list of all expected columns
expected_columns = factor_columns[:]
for ticker in ev_tickers_esg_components:
    for suffix in esg_suffixes:
        expected_columns.append(f"{ticker}{suffix}")

# Filter the loaded data
actual_columns_in_csv = [col for col in expected_columns if col in _ff_esg_data_raw.columns]
essential_factors = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'MOM', 'RF']
if not all(f in actual_columns_in_csv for f in essential_factors):
     missing_factors = [f for f in essential_factors if f not in actual_columns_in_csv]
     raise ValueError(f"Core factors missing: {missing_factors}")

ff_esg_data = _ff_esg_data_raw[actual_columns_in_csv].copy()
print(f"Using {len(ff_esg_data.columns)} columns from CSV.")

# Convert factor data index to datetime PeriodIndex (MonthEnd)
try:
    if not isinstance(ff_esg_data.index, pd.DatetimeIndex):
        raise TypeError("Index not DatetimeIndex after loading.")
    ff_esg_data.index = ff_esg_data.index.to_period('M').to_timestamp('M') # Ensure Month End
    ff_esg_data.dropna(axis=0, how='all', subset=factor_columns[:-1], inplace=True)
except Exception as e: raise ValueError(f"Error processing factor data index: {e}")

# --- Impute Missing ESG Data (Treat 0 as NaN before median imputation) ---
print("\nImputing missing ESG data (Aggregate & Components) using ticker-specific medians...")
imputation_count = 0
# Impute Aggregate ESG first, then components
for suffix in ['_ESG', '_E', '_S', '_G']:
    for ticker in ev_tickers_esg_components:
        col_name = f"{ticker}{suffix}"
        if col_name in ff_esg_data.columns:
            # Replace 0 with NaN
            ff_esg_data[col_name] = ff_esg_data[col_name].replace(0, np.nan)
            series = ff_esg_data[col_name]
            if series.isnull().any():
                median_val = series.median()
                if pd.isna(median_val):
                    print(f"Warning: Median for {col_name} is NaN. Filling missing/0 with 0.")
                    fill_value = 0
                else:
                    fill_value = median_val
                num_filled = series.isnull().sum()
                ff_esg_data[col_name] = series.fillna(fill_value)
                imputation_count += num_filled
print(f"ESG imputation complete. Attempted imputation on {imputation_count} values.")

# --- Data Preparation for Panel ---

# 1. Align Returns and Imputed Factor/ESG Data by Date
print("\nAligning returns and factor/ESG data by date...")
monthly_returns.index = monthly_returns.index.to_period('M').to_timestamp('M') # Ensure index matches
common_index_dates = monthly_returns.index.intersection(ff_esg_data.index)
if common_index_dates.empty: raise ValueError("No overlapping dates.")
start_date, end_date = common_index_dates.min(), common_index_dates.max()
print(f"Common date range: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
monthly_returns_aligned = monthly_returns.loc[common_index_dates]
ff_esg_data_aligned = ff_esg_data.loc[common_index_dates]

# 2. Prepare Dependent Variable (Excess Returns) in Long Format
print("Preparing excess returns in long format...")
excess_returns_long_list = []
rf_aligned = ff_esg_data_aligned["RF"]
for ticker in ev_tickers_returns:
    if ticker in monthly_returns_aligned.columns:
        temp_ret = pd.DataFrame({'Return': monthly_returns_aligned[ticker], 'Ticker': ticker}, index=monthly_returns_aligned.index)
        temp_ret = temp_ret.join(rf_aligned.rename('RF'), how='left')
        temp_ret['excess_return'] = temp_ret['Return'] - temp_ret['RF']
        excess_returns_long_list.append(temp_ret[['Ticker', 'excess_return']])
if not excess_returns_long_list: raise ValueError("No excess returns calculated.")
excess_returns_with_index = pd.concat(excess_returns_long_list)
excess_returns_long = excess_returns_with_index.reset_index().rename(columns={'Date': 'Date'})
excess_returns_long.dropna(subset=['excess_return'], inplace=True)

# 3. Prepare Imputed ESG Data (Aggregate, E, S, G) in Long Format
print("Preparing imputed ESG data (Aggregate, E, S, G) in long format...")
esg_data_long_list = []
for ticker in ev_tickers_esg_components:
    # Select relevant columns for this ticker (E,S,G and Aggregate)
    ticker_cols = {'Ticker': ticker}
    processed_any = False
    for suffix in ['_E', '_S', '_G', '_ESG']:
        col_name = f"{ticker}{suffix}"
        if col_name in ff_esg_data_aligned.columns:
            ticker_cols[suffix[1:]] = ff_esg_data_aligned[col_name] # Use short name E,S,G,ESG
            processed_any = True
        else:
            ticker_cols[suffix[1:]] = np.nan # Add NaN column if missing

    if processed_any:
         esg_data_long_list.append(pd.DataFrame(ticker_cols, index=ff_esg_data_aligned.index))

if not esg_data_long_list: print("Warning: No ESG data processed.")
esg_data_with_index = pd.concat(esg_data_long_list) if esg_data_long_list else pd.DataFrame(columns=['Ticker','E','S','G','ESG'])
esg_data_long = esg_data_with_index.reset_index().rename(columns={'Date': 'Date'})

# 4. Merge Returns, Factors, and ALL ESG Data
print("Merging returns, factors, and all ESG data...")
panel_data = pd.merge(excess_returns_long, ff_esg_data_aligned[factor_columns].reset_index().rename(columns={'Date':'Date'}), on='Date', how='left')
panel_data = pd.merge(panel_data, esg_data_long[['Date', 'Ticker', 'E', 'S', 'G', 'ESG']], on=['Date', 'Ticker'], how='left')

# 5. Create Lagged Variables
print("Creating lagged ESG variables (Aggregate and Components)...")
panel_data.sort_values(by=['Ticker', 'Date'], inplace=True)
lagged_esg_cols = ['E_lagged', 'S_lagged', 'G_lagged', 'ESG_lagged']
panel_data['E_lagged'] = panel_data.groupby('Ticker')['E'].shift(1)
panel_data['S_lagged'] = panel_data.groupby('Ticker')['S'].shift(1)
panel_data['G_lagged'] = panel_data.groupby('Ticker')['G'].shift(1)
panel_data['ESG_lagged'] = panel_data.groupby('Ticker')['ESG'].shift(1) # Lag aggregate score too

# 6. Handle Final Missing Values
print("Handling final missing values...")
essential_factors_no_rf = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'MOM'] # 6 factors
# Define essential cols needed across ALL models we plan to run
essential_cols = ['excess_return'] + lagged_esg_cols + essential_factors_no_rf
initial_rows = len(panel_data)
panel_data.dropna(subset=essential_cols, inplace=True) # Drop if ANY essential column is NA
rows_after_na = len(panel_data)
print(f"Dropped {initial_rows - rows_after_na} rows due to missing values.")
if panel_data.empty: raise ValueError("Panel data empty after handling NaNs.")

# --- Pre-Regression Diagnostics (Run BEFORE setting index) ---
print("\nChecking within-entity variance for lagged ESG components...")
within_variance = panel_data.groupby('Ticker')[lagged_esg_cols].var(ddof=0)
print("Within-Entity Variance:\n", within_variance)
low_variance_threshold = 1e-6 # Check for very low variance
low_variance_issues = {}
for col in lagged_esg_cols:
    if col in within_variance.columns: # Check if column exists after potential drops
        low_var_tickers = within_variance.index[within_variance[col] < low_variance_threshold].tolist()
        if low_var_tickers:
            low_variance_issues[col] = low_var_tickers
            print(f"WARNING: Low within-entity variance (<{low_variance_threshold}) for '{col}' in tickers: {low_var_tickers}.")

print("\nChecking correlations of demeaned lagged ESG components...")
try:
    if not all(col in panel_data.columns for col in lagged_esg_cols):
        raise KeyError("Lagged ESG columns missing before demeaning.")
    # Select only component columns for correlation check
    component_lagged_cols = ['E_lagged', 'S_lagged', 'G_lagged']
    demeaned_esg_components = panel_data.groupby('Ticker')[component_lagged_cols].transform(lambda x: x - x.mean())
    correlation_matrix_components = demeaned_esg_components.corr()
    print("Correlation Matrix (Demeaned E, S, G):\n", correlation_matrix_components)
    high_corr_threshold = 0.8
    mask = ((correlation_matrix_components > high_corr_threshold) | (correlation_matrix_components < -high_corr_threshold)) & (correlation_matrix_components != 1.0)
    high_corrs = correlation_matrix_components[mask].unstack().sort_values(ascending=False).drop_duplicates()
    if not high_corrs.empty:
         print(f"\nWARNING: High correlations (> {high_corr_threshold}) detected between demeaned lagged E,S,G components:")
         print(high_corrs)
except KeyError as ke: print(f"Error during demeaning/correlation check: {ke}")
except Exception as corr_err: print(f"Error calculating demeaned correlations: {corr_err}")

# --- Set Panel MultiIndex ---
print("\nSetting Panel MultiIndex (Ticker, Date)...")
try:
    panel_data['Date'] = pd.to_datetime(panel_data['Date'])
    panel_data = panel_data.set_index(['Ticker', 'Date'])
except KeyError: raise KeyError("Columns 'Ticker' or 'Date' not found.")
print(f"Final Panel Data Ready. Shape: {panel_data.shape}")

# --- Panel Regression Analysis ---
all_results = {} # Dictionary to store results

# --- Model 1: Separate Fixed Effects for E, S, G ---
print("\n--- Running Separate FE Models for E, S, G ---")
for component in ['E_lagged', 'S_lagged', 'G_lagged']:
    model_name = f"FE_{component.split('_')[0]}"
    print(f"\n--- {model_name} ---")
    try:
        dependent = panel_data['excess_return']
        exog_vars = [component]
        if not all(v in panel_data.columns for v in exog_vars): raise ValueError(f"{exog_vars} not found.")
        exog_df_final = panel_data[exog_vars]
        if exog_df_final.empty or exog_df_final.var(ddof=0).iloc[0] < 1e-9: raise ValueError(f"{component} has zero/low variance.")
        exog = exog_df_final
        mod = PanelOLS(dependent, exog, entity_effects=True, time_effects=True)
        fe_res = mod.fit(cov_type='clustered', cluster_entity=True, cluster_time=True)
        print(fe_res)
        all_results[model_name] = fe_res.summary
    except ValueError as ve: print(f"MODEL ESTIMATION ERROR for {model_name}: {ve}"); all_results[model_name] = f"Error: {ve}"
    except Exception as e: print(f"UNEXPECTED ERROR for {model_name}: {e}"); all_results[model_name] = f"Error: {e}"

# --- Model 2: Fixed Effects for Aggregate ESG ---
print("\n--- Running FE Model for Aggregate ESG ---")
model_name = "FE_ESG_Aggregate"
try:
    dependent = panel_data['excess_return']
    exog_vars = ['ESG_lagged'] # Use Aggregate ESG score
    if not all(v in panel_data.columns for v in exog_vars): raise ValueError(f"{exog_vars} not found.")
    exog_df_final = panel_data[exog_vars]
    if exog_df_final.empty or exog_df_final.var(ddof=0).iloc[0] < 1e-9: raise ValueError(f"ESG_lagged has zero/low variance.")
    exog = exog_df_final
    mod = PanelOLS(dependent, exog, entity_effects=True, time_effects=True)
    fe_agg_res = mod.fit(cov_type='clustered', cluster_entity=True, cluster_time=True)
    print(fe_agg_res)
    all_results[model_name] = fe_agg_res.summary
except ValueError as ve: print(f"MODEL ESTIMATION ERROR for {model_name}: {ve}"); all_results[model_name] = f"Error: {ve}"
except Exception as e: print(f"UNEXPECTED ERROR for {model_name}: {e}"); all_results[model_name] = f"Error: {e}"

# --- Model 3: Random Effects with E, S, G ---
print("\n--- Running Random Effects Model with E, S, G ---")
print("Note: RE assumes predictors uncorrelated with unobserved entity effects.")
model_key = "RE_ESG_Components"
try:
    panel_data_re = panel_data.reset_index() # Work with columns for RE + constant
    dependent_re = panel_data_re['excess_return']
    exog_vars_re = ['E_lagged', 'S_lagged', 'G_lagged']
    if not all(v in panel_data_re.columns for v in exog_vars_re): raise ValueError(f"{exog_vars_re} not found for RE.")
    exog_df_final_re = panel_data_re[exog_vars_re]
    if exog_df_final_re.empty or (exog_df_final_re.var(ddof=0) == 0).any(): raise ValueError("ESG components have no data/variance for RE.")
    # Add constant for RE
    exog_re = sm.add_constant(exog_df_final_re)
    # Need indexed dependent and entity/time columns for RE fitting
    panel_data_re_indexed = panel_data_re.set_index(['Ticker', 'Date'])
    mod_re = RandomEffects(panel_data_re_indexed['excess_return'], exog_re) # Pass columns including constant
    re_res = mod_re.fit(cov_type='clustered', cluster_entity=True, cluster_time=True)
    print(re_res)
    all_results[model_key] = re_res.summary
except ValueError as ve: print(f"MODEL ESTIMATION ERROR for RE: {ve}"); all_results[model_key] = f"Error: {ve}"
except Exception as e: print(f"UNEXPECTED ERROR for RE: {e}"); all_results[model_key] = f"Error: {e}"

# --- Save All Results to Excel ---
output_filename = 'panel_regression_results_long_series_strategies.xlsx' # Updated filename
print(f"\nSaving all regression results to '{output_filename}'...")
try:
    with pd.ExcelWriter(output_filename, engine='xlsxwriter') as writer:
        for model_name, summary_obj in all_results.items():
            print(f"Saving results for {model_name}...")
            if isinstance(summary_obj, str): # Error message
                pd.DataFrame({'Error': [summary_obj]}).to_excel(writer, sheet_name=f'{model_name}_Error', index=False)
            elif hasattr(summary_obj, 'tables') and len(summary_obj.tables) >= 3:
                try:
                    df_top = pd.read_html(StringIO(summary_obj.tables[0].as_html()), header=None, index_col=0)[0]
                    df_coeffs = pd.read_html(StringIO(summary_obj.tables[1].as_html()), header=0, index_col=0)[0]
                    df_bottom = pd.read_html(StringIO(summary_obj.tables[2].as_html()), header=None, index_col=0)[0]
                    df_top.to_excel(writer, sheet_name=f'{model_name}_Model', header=False)
                    df_coeffs.reset_index().to_excel(writer, sheet_name=f'{model_name}_Coefs', index=False)
                    df_bottom.to_excel(writer, sheet_name=f'{model_name}_Diag', header=False)
                except Exception as e_parse:
                    print(f"  Error parsing/saving tables for {model_name}: {e_parse}")
                    pd.DataFrame({'Error': [f"Error parsing/saving tables: {e_parse}"]}).to_excel(writer, sheet_name=f'{model_name}_Error', index=False)
            else:
                print(f"  Warning: Summary object format unexpected for {model_name}.")
                pd.DataFrame({'Error': ["Summary object format unexpected."]}).to_excel(writer, sheet_name=f'{model_name}_Error', index=False)
    print(f"All attempted model results saved to '{output_filename}'.")
except Exception as save_err:
    print(f"ERROR: Could not write to Excel file '{output_filename}': {save_err}")

Stock data download complete.

Calculating monthly returns...
Monthly returns calculated.

Loading Fama-French 6-Factor and ESG data from '/content/gd_Developed_5_Factors.csv'...
Data loaded successfully.
Using 55 columns from CSV.

Imputing missing ESG data (Aggregate & Components) using ticker-specific medians...
ESG imputation complete. Attempted imputation on 2246 values.

Aligning returns and factor/ESG data by date...
Common date range: 2019-12-31 to 2024-12-31
Preparing excess returns in long format...
Preparing imputed ESG data (Aggregate, E, S, G) in long format...
Merging returns, factors, and all ESG data...
Creating lagged ESG variables (Aggregate and Components)...
Handling final missing values...
Dropped 12 rows due to missing values.

Checking within-entity variance for lagged ESG components...
Within-Entity Variance:
             E_lagged   S_lagged   G_lagged  ESG_lagged
Ticker                                                
005380.KS  42.572805  22.404980  23.985233  

/usr/local/lib/python3.11/dist-packages/linearmodels/panel/results.py:89: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")


                          PanelOLS Estimation Summary                           
Dep. Variable:          excess_return   R-squared:                     3.527e-05
Estimator:                   PanelOLS   R-squared (Between):              0.0516
No. Observations:                 651   R-squared (Within):             4.96e-05
Date:                Fri, Mar 28 2025   R-squared (Overall):              0.0015
Time:                        21:13:15   Log-likelihood                   -2775.7
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      0.0204
Entities:                          12   P-value                           0.8864
Avg Obs:                       54.250   Distribution:                   F(1,579)
Min Obs:                       36.000                                           
Max Obs:                       60.000   F-statistic (robust):            -0.7728
                            

In [62]:
import yesg
import pandas as pd
import numpy as np
import time

# List of EV tickers
ev_tickers = ["TSLA", "BYDDY", "VWAGY", "NIO", "RIVN",
               "LCID", "XPEV", "LI", "GM", "F", "005380.KS", "PSNY", "BMW.DE", "TM", "MBG.DE"] # 005380.KS

# Initialize an empty DataFrame to hold the merged ESG data
merged_esg_data = pd.DataFrame()

# Function to fetch ESG data with retries
def fetch_esg_data(ticker, retries=3):
    for attempt in range(retries):
        esg_data = yesg.get_historic_esg(ticker)
        if esg_data is not None:
            return esg_data
        else:
            print(f"Attempt {attempt + 1} failed for {ticker}. Retrying...")
            time.sleep(2)  # Wait before retrying
    print(f"No ESG data found for {ticker} after {retries} attempts.")
    return None

# Loop through each ticker and fetch ESG scores
for ticker in ev_tickers:
    esg_data = fetch_esg_data(ticker)

    if esg_data is None:
        continue  # Skip to the next ticker if no data is found

    # Rename columns based on the ticker
    esg_data.columns = [f"{ticker}_ESG", f"{ticker}_E", f"{ticker}_S", f"{ticker}_G"]

    # Fill NA values with 0
    esg_data.replace(np.nan, 0, inplace=True)

    # Merge the current ESG data with the merged DataFrame
    if merged_esg_data.empty:
        merged_esg_data = esg_data
    else:
        merged_esg_data = merged_esg_data.join(esg_data, how='outer')

# Save the merged DataFrame to a CSV file
csv_file_path = '/content/all_ev_esg.csv'
merged_esg_data.to_csv(csv_file_path, index=False)

# Display the last few rows of the merged DataFrame
print(merged_esg_data.tail(3))
print(f"CSV saved to: {csv_file_path}")

An error has occurred. The ticker symbol might be wrong or you might need to wait to continue.
Attempt 1 failed for BYDDY. Retrying...
An error has occurred. The ticker symbol might be wrong or you might need to wait to continue.
Attempt 2 failed for BYDDY. Retrying...
An error has occurred. The ticker symbol might be wrong or you might need to wait to continue.
Attempt 3 failed for BYDDY. Retrying...
No ESG data found for BYDDY after 3 attempts.
An error has occurred. The ticker symbol might be wrong or you might need to wait to continue.
Attempt 1 failed for VWAGY. Retrying...
An error has occurred. The ticker symbol might be wrong or you might need to wait to continue.
Attempt 2 failed for VWAGY. Retrying...
An error has occurred. The ticker symbol might be wrong or you might need to wait to continue.
Attempt 3 failed for VWAGY. Retrying...
No ESG data found for VWAGY after 3 attempts.
An error has occurred. The ticker symbol might be wrong or you might need to wait to continue.
Att

In [58]:
# --- Panel Regression Analysis ---
all_results = {} # Dictionary to store results

# Make sure panel_data is indexed correctly
if not isinstance(panel_data.index, pd.MultiIndex) or \
   not all(level in panel_data.index.names for level in ['Ticker', 'Date']):
    try:
        print("Setting Panel MultiIndex (Ticker, Date) before running models...")
        panel_data = panel_data.set_index(['Ticker', 'Date'])
    except KeyError:
        raise KeyError("Columns 'Ticker' or 'Date' not found before setting index.")


# --- Run Separate Fixed Effects Models for E, S, G ---
print("\n--- Running Separate FE Models for E, S, G ---")
lagged_esg_cols = ['E_lagged', 'S_lagged', 'G_lagged']

for component in lagged_esg_cols:
    model_name = f"FE_{component.split('_')[0]}" # e.g., FE_E
    print(f"\n--- {model_name} ---")
    try:
        # Check for component existence and variance again just before model run
        if component not in panel_data.columns:
            raise ValueError(f"Predictor '{component}' not found in final panel_data.")

        exog_df = panel_data[[component]] # Select only the single component
        dependent = panel_data['excess_return']

        # Check variance for this specific component in the final data
        if exog_df.var(ddof=0).iloc[0] < 1e-9: # Use a slightly higher threshold for check
             raise ValueError(f"'{component}' has zero or near-zero variance in the final dataset ({exog_df.var(ddof=0).iloc[0]:.2e}). Cannot estimate FE model.")

        # Check degrees of freedom for single predictor FE model
        n_obs = exog_df.shape[0]
        n_entity = panel_data.index.get_level_values(0).nunique()
        n_time = panel_data.index.get_level_values(1).nunique()
        n_exog = 1 # Only one predictor
        df_resid = n_obs - n_entity - n_time - n_exog + 1
        if df_resid <= 0:
            raise ValueError(f"Insufficient observations ({n_obs}) for degrees of freedom ({df_resid}) for {component}.")

        # Define and Fit Model
        mod = PanelOLS(dependent, exog_df, entity_effects=True, time_effects=True)
        fe_res = mod.fit(cov_type='clustered', cluster_entity=True, cluster_time=True)

        # Print and Store Results
        print(fe_res)
        all_results[model_name] = fe_res.summary

    except ValueError as ve:
        print(f"MODEL ESTIMATION ERROR for {model_name}: {ve}")
        all_results[model_name] = f"Error: {ve}"
    except Exception as e:
        print(f"UNEXPECTED ERROR for {model_name}: {e}")
        all_results[model_name] = f"Error: {e}"

# --- Optional: Include RE Model for comparison (as before) ---
# ...(Code for RE model can be added here if desired)...

# --- Optional: Include PCA Model for comparison (as before) ---
# ...(Code for PCA FE model can be added here if desired)...


# --- Save All Attempted Results to Excel ---
output_filename = 'panel_regression_results_separate_FE.xlsx' # Updated filename
print(f"\nSaving all regression results to '{output_filename}'...")
try:
    with pd.ExcelWriter(output_filename, engine='xlsxwriter') as writer:
        for model_name, summary_obj in all_results.items():
            print(f"Saving results for {model_name}...")
            if isinstance(summary_obj, str): # Error message
                pd.DataFrame({'Error': [summary_obj]}).to_excel(writer, sheet_name=f'{model_name}_Error', index=False)
            elif hasattr(summary_obj, 'tables') and len(summary_obj.tables) >= 3:
                # ... (rest of saving code using StringIO as before) ...
                try:
                    df_top = pd.read_html(StringIO(summary_obj.tables[0].as_html()), header=None, index_col=0)[0]
                    df_coeffs = pd.read_html(StringIO(summary_obj.tables[1].as_html()), header=0, index_col=0)[0]
                    df_bottom = pd.read_html(StringIO(summary_obj.tables[2].as_html()), header=None, index_col=0)[0]
                    df_top.to_excel(writer, sheet_name=f'{model_name}_Model', header=False)
                    df_coeffs.reset_index().to_excel(writer, sheet_name=f'{model_name}_Coefs', index=False)
                    df_bottom.to_excel(writer, sheet_name=f'{model_name}_Diag', header=False)
                except Exception as e_parse:
                     print(f"  Error parsing/saving tables for {model_name}: {e_parse}")
                     pd.DataFrame({'Error': [f"Error parsing/saving tables: {e_parse}"]}).to_excel(writer, sheet_name=f'{model_name}_Error', index=False)
            else:
                print(f"  Warning: Summary object format unexpected for {model_name}.")
                pd.DataFrame({'Error': ["Summary object format unexpected."]}).to_excel(writer, sheet_name=f'{model_name}_Error', index=False)
    print(f"All attempted model results saved to '{output_filename}'.")
except Exception as save_err:
    print(f"ERROR: Could not write to Excel file '{output_filename}': {save_err}")


--- Running Separate FE Models for E, S, G ---

--- FE_E ---
                          PanelOLS Estimation Summary                           
Dep. Variable:          excess_return   R-squared:                     2.876e-05
Estimator:                   PanelOLS   R-squared (Between):             -0.1273
No. Observations:                 420   R-squared (Within):            6.034e-05
Date:                Fri, Mar 28 2025   R-squared (Overall):             -0.0016
Time:                        18:44:35   Log-likelihood                   -1749.4
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      0.0107
Entities:                          12   P-value                           0.9176
Avg Obs:                       35.000   Distribution:                   F(1,373)
Min Obs:                       35.000                                           
Max Obs:                       35.000   F-stati

In [61]:

import yesg
import pandas as pd
import numpy as np


# All available historic ESG rating for THE Critical Component COMPANY
TSLA_ev = yesg.get_historic_esg('TSLA')
TSLA_ev.columns = ['Guan_ESG', 'Guan_E', 'Guan_S', 'Guang_G']

# Fill na with 0
TSLA_ev = pd.DataFrame(TSLA_ev)

# inplace
TSLA_ev.replace(np.nan, 0, inplace=True)

# Guangzhou_high_ev.tail()


# Merge and save as csv
#eighteenth_high_ev = seventeenth_high_ev.join(Guangzhou_high_ev, how='outer')

# Save to csv
#all_low_bat5.to_csv(r'/Users/henryefeonomakpo/Downloads/1-Indra-H-Thesis idea/1-saved csv from python/all18_ev.csv')

#display
display(TSLA_ev)

,Guan_ESG,Guan_E,Guan_S,Guang_G
Date,,,,
2014-09-01,56.00,62.00,50.00,52.0
2014-10-01,56.00,62.00,50.00,52.0
2014-11-01,56.00,62.00,50.00,52.0
2014-12-01,56.00,62.00,50.00,52.0
2015-01-01,56.00,62.00,50.00,55.0
...,...,...,...,...
2024-11-01,24.73,3.20,14.13,7.4
2024-12-01,24.73,3.20,14.13,7.4
2025-01-01,24.73,3.20,14.13,7.4


### Scrape ESG risk rating dataset

In [57]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.style as style
import yfinance as yf
import statsmodels.api as sm
import numpy as np
import xlsxwriter
import warnings
from io import StringIO

# Import PanelOLS and RandomEffects from linearmodels
from linearmodels.panel import PanelOLS, RandomEffects

# Import PCA and StandardScaler from sklearn
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# --- Standard Setup (Warnings, Style) ---
warnings.filterwarnings("ignore", category=FutureWarning) # Suppress various FutureWarnings

# Set plotting style (optional)
style.use("default")
params = {"axes.labelsize": 8, "font.size": 8, "legend.fontsize": 8, "xtick.labelsize": 8,
          "ytick.labelsize": 8, "text.usetex": False, "font.family": "sans-serif",
          "axes.spines.top": False, "axes.spines.right": False, "grid.color": "grey",
          "axes.grid": True, "grid.alpha": 0.5, "grid.linestyle": ":"}
plt.rcParams.update(params)

# --- Constants ---
ev_tickers_returns = ["TSLA", "1211.HK", "VOW3.DE", "NIO", "RIVN", "LCID", "XPEV",
                      "LI", "GM", "F", "005380.KS", "PSNY", "BMW.DE"]
ev_tickers_esg_components = ["TSLA", "1211.HK", "VOW3.DE", "NIO", "RIVN", "LCID", "XPEV",
                             "LI", "GM", "F", "BMW.DE", "PSNY"] # Tickers expected to have E,S,G cols
csv_file_path = "/content/gd_Developed_5_Factors.csv" # MAKE SURE THIS PATH IS CORRECT

# --- Data Downloading (Returns) ---
adj_close_data = pd.DataFrame()
print("Downloading historical stock data...")
download_start_date = "2021-11-01" # Start earlier for lagging
download_end_date = "2025-03-27"
for ticker in ev_tickers_returns:
    try:
        data = yf.download(ticker, start=download_start_date, end=download_end_date, progress=False)
        if data.empty:
            print(f"No data downloaded for {ticker}. Skipping.")
            continue
        adj_close_data[ticker] = data.get("Adj Close", data["Close"])
        # print(f"Successfully downloaded data for {ticker}.") # Keep less verbose
    except Exception as e:
        print(f"Error downloading data for {ticker}: {e}")
print("Stock data download complete.")

# --- Calculate Monthly Returns ---
print("\nCalculating monthly returns...")
if adj_close_data.empty: raise ValueError("No stock data downloaded.")
monthly_returns = adj_close_data.resample("ME").last().pct_change() * 100
print("Monthly returns calculated.")

# --- Load Fama-French 6-Factor and ESG data ---
print(f"\nLoading Fama-French 6-Factor and ESG data from '{csv_file_path}'...")
try:
    _ff_esg_data_raw = pd.read_csv(csv_file_path, index_col=0, header=0, on_bad_lines='warn')
    print("Data loaded successfully.")
except FileNotFoundError: raise FileNotFoundError(f"CSV file not found: {csv_file_path}")
except Exception as e: raise Exception(f"Error loading CSV file: {e}")

# Define expected factor columns (FF5 + MOM + RF)
factor_columns = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF', 'MOM']
esg_suffixes = ['_E', '_S', '_G', '_ESG'] # Base suffixes to look for

# Build list of all expected columns
expected_columns = factor_columns[:]
for ticker in ev_tickers_esg_components:
    for suffix in esg_suffixes:
        expected_columns.append(f"{ticker}{suffix}")

# Filter the loaded data
actual_columns_in_csv = [col for col in expected_columns if col in _ff_esg_data_raw.columns]
essential_factors = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'MOM', 'RF']
if not all(f in actual_columns_in_csv for f in essential_factors):
     missing_factors = [f for f in essential_factors if f not in actual_columns_in_csv]
     raise ValueError(f"Core Fama-French/Momentum factors missing: {missing_factors}")

ff_esg_data = _ff_esg_data_raw[actual_columns_in_csv].copy()
print(f"Using {len(ff_esg_data.columns)} columns from CSV.")

# Convert factor data index to datetime
try:
    ff_esg_data.index = pd.to_datetime(ff_esg_data.index.astype(str), format="%Y%m", errors='coerce')
    ff_esg_data.index = ff_esg_data.index + pd.offsets.MonthEnd(0)
    ff_esg_data.dropna(axis=0, how='all', subset=factor_columns[:-1], inplace=True)
except ValueError as e: raise ValueError(f"Error converting factor data index to datetime: {e}")


# --- Impute Missing ESG Component Data (BEFORE Stacking/Merging) ---
print("\nImputing missing ESG component data (E, S, G) using ticker-specific medians...")
imputation_count = 0
for ticker in ev_tickers_esg_components:
    for suffix in ['_E', '_S', '_G']:
        col_name = f"{ticker}{suffix}"
        if col_name in ff_esg_data.columns:
            series = ff_esg_data[col_name]
            if series.isnull().any():
                median_val = series.median()
                if pd.isna(median_val):
                    print(f"Warning: Median for {col_name} is NaN. Filling missing values with 0.")
                    fill_value = 0
                else:
                    fill_value = median_val
                num_filled = series.isnull().sum()
                ff_esg_data[col_name] = series.fillna(fill_value)
                imputation_count += num_filled
print(f"ESG component imputation complete. Imputed {imputation_count} values.")

# --- Data Preparation for Panel ---

# 1. Align Returns and Imputed Factor/ESG Data by Date
print("\nAligning returns and factor/ESG data by date...")
common_index_dates = monthly_returns.index.intersection(ff_esg_data.index)
if common_index_dates.empty: raise ValueError("No overlapping dates found after loading/imputing.")
start_date, end_date = common_index_dates.min(), common_index_dates.max()
print(f"Common date range for panel: {start_date.date()} to {end_date.date()}")
monthly_returns_aligned = monthly_returns.loc[common_index_dates]
ff_esg_data_aligned = ff_esg_data.loc[common_index_dates]

# 2. Prepare Dependent Variable (Excess Returns) in Long Format
print("Preparing excess returns in long format...")
excess_returns_long_list = []
rf_aligned = ff_esg_data_aligned["RF"]
for ticker in ev_tickers_returns:
    if ticker in monthly_returns_aligned.columns:
        temp_ret = pd.DataFrame({'Return': monthly_returns_aligned[ticker], 'Ticker': ticker}, index=monthly_returns_aligned.index)
        temp_ret = temp_ret.join(rf_aligned.rename('RF'), how='left')
        temp_ret['excess_return'] = temp_ret['Return'] - temp_ret['RF']
        excess_returns_long_list.append(temp_ret[['Ticker', 'excess_return']])
if not excess_returns_long_list: raise ValueError("No excess returns calculated.")
excess_returns_with_index = pd.concat(excess_returns_long_list)
excess_returns_long = excess_returns_with_index.reset_index().rename(columns={'index': 'Date'})
excess_returns_long.dropna(subset=['excess_return'], inplace=True)

# 3. Prepare Imputed ESG Component Data (E, S, G) in Long Format
print("Preparing imputed ESG component data (E, S, G) in long format...")
esg_components_long_list = []
for ticker in ev_tickers_esg_components:
    component_data_df = pd.DataFrame({'Ticker': ticker}, index=ff_esg_data_aligned.index)
    processed_any_component = False
    for suffix in ['_E', '_S', '_G']:
        col_name = f"{ticker}{suffix}"
        if col_name in ff_esg_data_aligned.columns:
            component_data_df[suffix[1:]] = ff_esg_data_aligned[col_name]
            processed_any_component = True
        else: component_data_df[suffix[1:]] = np.nan
    if processed_any_component: esg_components_long_list.append(component_data_df)
if not esg_components_long_list: print("Warning: No ESG component data processed.")
esg_components_with_index = pd.concat(esg_components_long_list) if esg_components_long_list else pd.DataFrame(columns=['Ticker','E','S','G'])
esg_components_long = esg_components_with_index.reset_index().rename(columns={'index': 'Date'})

# 4. Merge Returns, Factors, and ESG Components
print("Merging returns, factors, and ESG component data...")
panel_data = pd.merge(excess_returns_long, ff_esg_data_aligned[factor_columns].reset_index().rename(columns={'index':'Date'}), on='Date', how='left')
panel_data = pd.merge(panel_data, esg_components_long[['Date', 'Ticker', 'E', 'S', 'G']], on=['Date', 'Ticker'], how='left')

# 5. Create Lagged ESG Component Variables
print("Creating lagged ESG component variables...")
panel_data.sort_values(by=['Ticker', 'Date'], inplace=True)
lagged_esg_cols = ['E_lagged', 'S_lagged', 'G_lagged']
panel_data['E_lagged'] = panel_data.groupby('Ticker')['E'].shift(1)
panel_data['S_lagged'] = panel_data.groupby('Ticker')['S'].shift(1)
panel_data['G_lagged'] = panel_data.groupby('Ticker')['G'].shift(1)

# 6. Handle Final Missing Values
print("Handling final missing values after merging and lagging...")
essential_factors_no_rf = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'MOM'] # 6 factors
essential_cols = ['excess_return'] + lagged_esg_cols + essential_factors_no_rf
initial_rows = len(panel_data)
# Drop rows where ANY essential variable for ANY model is missing
panel_data.dropna(subset=essential_cols, inplace=True)
rows_after_na = len(panel_data)
print(f"Dropped {initial_rows - rows_after_na} rows due to missing values.")
if panel_data.empty: raise ValueError("Panel data empty after handling NaNs.")

# --- Prepare for PCA (Run BEFORE setting index) ---
# Uses the final cleaned panel_data
print("\nPreparing data for PCA analysis...")
pca_data_final = panel_data[lagged_esg_cols].copy()
if pca_data_final.isnull().any().any():
    print("Warning: NaNs found before PCA. Applying mean imputation...")
    pca_data_final = pca_data_final.fillna(pca_data_final.mean())
print("Scaling data for PCA...")
scaler = StandardScaler()
scaled_data = scaler.fit_transform(pca_data_final)
print("Running PCA...")
pca = PCA(n_components=3)
pca_components = pca.fit_transform(scaled_data)
print("Explained Variance Ratio by PCA components:", pca.explained_variance_ratio_)
panel_data[['PC1', 'PC2', 'PC3']] = pca_components # Add PCA components to main dataframe
pca_cols = ['PC1', 'PC2', 'PC3']

# --- Set Panel MultiIndex ---
print("\nSetting Panel MultiIndex (Ticker, Date)...")
try:
    panel_data = panel_data.set_index(['Ticker', 'Date'])
except KeyError: raise KeyError("Columns 'Ticker' or 'Date' not found.")
print(f"Final Panel Data Ready. Shape: {panel_data.shape}")

# --- Panel Regression Analysis ---
all_results = {} # Dictionary to store results

# --- Model 1: Separate Fixed Effects for E, S, G ---
print("\n--- Running Separate FE Models for E, S, G ---")
for component in lagged_esg_cols: # ['E_lagged', 'S_lagged', 'G_lagged']
    print(f"\n--- FE Model: {component} ---")
    model_key = f"FE_{component.split('_')[0]}"
    try:
        dependent = panel_data['excess_return']
        exog_vars = [component]
        if not all(v in panel_data.columns for v in exog_vars): raise ValueError(f"{exog_vars} not found.")
        exog_df_final = panel_data[exog_vars]
        if exog_df_final.empty or exog_df_final.var(ddof=0).iloc[0] == 0: raise ValueError(f"{component} has no data or no variance.")

        exog = exog_df_final
        mod = PanelOLS(dependent, exog, entity_effects=True, time_effects=True)
        fe_res = mod.fit(cov_type='clustered', cluster_entity=True, cluster_time=True)
        print(fe_res)
        all_results[model_key] = fe_res.summary

    except ValueError as ve: print(f"MODEL ESTIMATION ERROR for {component}: {ve}"); all_results[model_key] = f"Error: {ve}"
    except Exception as e: print(f"UNEXPECTED ERROR for {component}: {e}"); all_results[model_key] = f"Error: {e}"

# --- Model 2: Fixed Effects with PCA Components ---
print("\n--- Running FE Model with PCA Components ---")
model_key = "FE_PCA"
try:
    dependent = panel_data['excess_return']
    exog_vars = pca_cols # ['PC1', 'PC2', 'PC3']
    if not all(v in panel_data.columns for v in exog_vars): raise ValueError(f"PCA columns not found.")
    exog_df_final = panel_data[exog_vars]
    if exog_df_final.empty or (exog_df_final.var(ddof=0) == 0).any(): raise ValueError("PCA components have no data or variance.")

    exog = exog_df_final
    mod = PanelOLS(dependent, exog, entity_effects=True, time_effects=True)
    fe_pca_res = mod.fit(cov_type='clustered', cluster_entity=True, cluster_time=True)
    print(fe_pca_res)
    all_results[model_key] = fe_pca_res.summary

except ValueError as ve: print(f"MODEL ESTIMATION ERROR for PCA: {ve}"); all_results[model_key] = f"Error: {ve}"
except Exception as e: print(f"UNEXPECTED ERROR for PCA: {e}"); all_results[model_key] = f"Error: {e}"

# --- Model 3: Random Effects with E, S, G ---
print("\n--- Running Random Effects Model with E, S, G ---")
print("Note: RE assumes predictors uncorrelated with unobserved entity effects.")
model_key = "RE_ESG_Components"
try:
    dependent = panel_data['excess_return']
    exog_vars = ['E_lagged', 'S_lagged', 'G_lagged']
    if not all(v in panel_data.columns for v in exog_vars): raise ValueError(f"{exog_vars} not found for RE.")
    exog_df_final = panel_data[exog_vars]
    if exog_df_final.empty or (exog_df_final.var(ddof=0) == 0).any(): raise ValueError("ESG components have no data/variance for RE.")

    # Add constant for RE
    exog = sm.add_constant(exog_df_final)

    mod_re = RandomEffects(dependent, exog)
    re_res = mod_re.fit(cov_type='clustered', cluster_entity=True, cluster_time=True)
    print(re_res)
    all_results[model_key] = re_res.summary

except ValueError as ve: print(f"MODEL ESTIMATION ERROR for RE: {ve}"); all_results[model_key] = f"Error: {ve}"
except Exception as e: print(f"UNEXPECTED ERROR for RE: {e}"); all_results[model_key] = f"Error: {e}"


# --- Save All Results to Excel ---
output_filename = 'panel_regression_results_revised_strategies.xlsx' # Updated filename
print(f"\nSaving all regression results to '{output_filename}'...")
try:
    with pd.ExcelWriter(output_filename, engine='xlsxwriter') as writer:
        for model_name, summary_obj in all_results.items():
            print(f"Saving results for {model_name}...")
            if isinstance(summary_obj, str): # Error message
                pd.DataFrame({'Error': [summary_obj]}).to_excel(writer, sheet_name=f'{model_name}_Error', index=False)
            elif hasattr(summary_obj, 'tables') and len(summary_obj.tables) >= 3:
                try:
                    df_top = pd.read_html(StringIO(summary_obj.tables[0].as_html()), header=None, index_col=0)[0]
                    df_coeffs = pd.read_html(StringIO(summary_obj.tables[1].as_html()), header=0, index_col=0)[0]
                    df_bottom = pd.read_html(StringIO(summary_obj.tables[2].as_html()), header=None, index_col=0)[0]

                    df_top.to_excel(writer, sheet_name=f'{model_name}_Model', header=False)
                    df_coeffs.reset_index().to_excel(writer, sheet_name=f'{model_name}_Coefs', index=False)
                    df_bottom.to_excel(writer, sheet_name=f'{model_name}_Diag', header=False)
                except Exception as e_parse:
                    print(f"  Error parsing/saving tables for {model_name}: {e_parse}")
                    pd.DataFrame({'Error': [f"Error parsing/saving tables: {e_parse}"]}).to_excel(writer, sheet_name=f'{model_name}_Error', index=False)
            else:
                print(f"  Warning: Summary object format unexpected for {model_name}.")
                pd.DataFrame({'Error': ["Summary object format unexpected."]}).to_excel(writer, sheet_name=f'{model_name}_Error', index=False)
    print(f"All attempted model results saved to '{output_filename}'.")
except Exception as save_err:
    print(f"ERROR: Could not write to Excel file '{output_filename}': {save_err}")

Stock data download complete.

Calculating monthly returns...
Monthly returns calculated.

Loading Fama-French 6-Factor and ESG data from '/content/gd_Developed_5_Factors.csv'...
Data loaded successfully.
Using 55 columns from CSV.

Imputing missing ESG component data (E, S, G) using ticker-specific medians...
ESG component imputation complete. Imputed 0 values.

Aligning returns and factor/ESG data by date...
Common date range for panel: 2022-01-31 to 2024-12-31
Preparing excess returns in long format...
Preparing imputed ESG component data (E, S, G) in long format...
Merging returns, factors, and ESG component data...
Creating lagged ESG component variables...
Handling final missing values after merging and lagging...
Dropped 48 rows due to missing values.

Preparing data for PCA analysis...
Scaling data for PCA...
Running PCA...
Explained Variance Ratio by PCA components: [0.69397629 0.20844301 0.09758071]

Setting Panel MultiIndex (Ticker, Date)...
Final Panel Data Ready. Shape: (4

/usr/local/lib/python3.11/dist-packages/linearmodels/panel/results.py:89: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")


                          PanelOLS Estimation Summary                           
Dep. Variable:          excess_return   R-squared:                        0.0009
Estimator:                   PanelOLS   R-squared (Between):             -0.3215
No. Observations:                 420   R-squared (Within):               0.0006
Date:                Fri, Mar 28 2025   R-squared (Overall):             -0.0037
Time:                        18:37:51   Log-likelihood                   -1749.2
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      0.1172
Entities:                          12   P-value                           0.9500
Avg Obs:                       35.000   Distribution:                   F(3,371)
Min Obs:                       35.000                                           
Max Obs:                       35.000   F-statistic (robust):            -0.7652
                            

# Panel Data model (specifically, a Two-Way Fixed Effects model using the linearmodels library) with the Fama-French 5 factors and lagged aggregate ESG scores.
### Fetch EV Automakers ESG risk rating data using Python's yesg library

In [49]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.style as style
import yfinance as yf
import statsmodels.api as sm # Still useful for add_constant if needed outside PanelOLS
import numpy as np
# from statsmodels.stats.outliers_influence import variance_inflation_factor # VIF less standard/easy in Panel Fixed Effects
import xlsxwriter
import warnings # Import warnings module
from io import StringIO # Import StringIO

# Import PanelOLS from linearmodels
from linearmodels.panel import PanelOLS

# --- Standard Setup (Warnings, Style) ---
# Suppress common FutureWarnings
warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
    message="A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.*"
)
warnings.filterwarnings("ignore", category=FutureWarning, message="Passing literal html to 'read_html' is deprecated*")
warnings.filterwarnings("ignore", category=FutureWarning, message="'M' is deprecated*") # Handles yfinance 'M' -> 'ME' warning

# Optional: Suppress RuntimeWarning about divide by zero sometimes seen in statsmodels VIF (less relevant now)
# warnings.filterwarnings("ignore", category=RuntimeWarning, message="divide by zero encountered in scalar divide")

# Set plotting style (optional)
style.use("default")
params = {
    "axes.labelsize": 8, "font.size": 8, "legend.fontsize": 8,
    "xtick.labelsize": 8, "ytick.labelsize": 8, "text.usetex": False,
    "font.family": "sans-serif", "axes.spines.top": False, "axes.spines.right": False,
    "grid.color": "grey", "axes.grid": True, "grid.alpha": 0.5, "grid.linestyle": ":",
}
plt.rcParams.update(params)

# --- Constants ---
# List of EV tickers for returns download
ev_tickers_returns = ["TSLA", "1211.HK", "VOW3.DE", "NIO", "RIVN",
                      "LCID", "XPEV", "LI", "GM", "F", "005380.KS", "PSNY", "BMW.DE"]

# List of tickers corresponding to ESG columns in your FF5 CSV file
ev_tickers_esg = ["TSLA", "1211.HK", "VOW3.DE", "NIO", "RIVN",
                  "LCID", "XPEV", "LI", "GM", "F", "BMW.DE", "PSNY"] # 005380.KS excluded

# Path to the Fama-French 5-Factor + ESG data file
csv_file_path = "/content/gd_Developed_5_Factors.csv"

# --- Data Downloading (Returns) ---
adj_close_data = pd.DataFrame()
print("Downloading historical stock data...")
download_start_date = "2021-11-01"
download_end_date = "2025-03-27"
for ticker in ev_tickers_returns:
    try:
        data = yf.download(ticker, start=download_start_date, end=download_end_date, progress=False)
        if data.empty:
            print(f"No data downloaded for {ticker}. Skipping.")
            continue
        adj_close_data[ticker] = data.get("Adj Close", data["Close"])
        print(f"Successfully downloaded data for {ticker}.")
    except Exception as e:
        print(f"Error downloading data for {ticker}: {e}")

# --- Calculate Monthly Returns ---
print("\nCalculating monthly returns...")
if adj_close_data.empty:
    raise ValueError("No stock data was successfully downloaded.")
monthly_returns = adj_close_data.resample("ME").last().pct_change() * 100
print("Monthly returns calculated.")

# --- Load Fama-French 5-Factor and ESG data ---
print(f"\nLoading Fama-French 5-Factor and ESG data from '{csv_file_path}'...")
try:
    _ff_esg_data_raw = pd.read_csv(csv_file_path, index_col=0, header=0, on_bad_lines='warn')
    print("Data loaded successfully.")
except FileNotFoundError:
    raise FileNotFoundError(f"Error: The CSV file '{csv_file_path}' was not found.")
except Exception as e:
    raise Exception(f"Error loading CSV file: {e}")

# Define expected factor/ESG columns
factor_columns = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF']
esg_columns_expected = [f"{ticker}_ESG" for ticker in ev_tickers_esg]
expected_columns = factor_columns + esg_columns_expected

actual_columns_in_csv = [col for col in expected_columns if col in _ff_esg_data_raw.columns]
if not all(f in actual_columns_in_csv for f in factor_columns):
     raise ValueError(f"Core Fama-French factors missing in the loaded CSV.")
ff_esg_data = _ff_esg_data_raw[actual_columns_in_csv].copy()
print(f"Using columns from CSV: {list(ff_esg_data.columns)}")

# Convert factor data index to datetime
try:
    ff_esg_data.index = pd.to_datetime(ff_esg_data.index.astype(str), format="%Y%m", errors='coerce')
    ff_esg_data.index = ff_esg_data.index + pd.offsets.MonthEnd(0)
    ff_esg_data.dropna(axis=0, how='all', subset=factor_columns, inplace=True)
except ValueError as e:
    raise ValueError(f"Error converting factor data index to datetime: {e}")


# --- Data Preparation for Panel ---

# 1. Align Returns and Factor Data by Date
print("\nAligning returns and factor data by date...")
common_index_dates = monthly_returns.index.intersection(ff_esg_data.index)
if common_index_dates.empty:
    raise ValueError("No overlapping dates found between stock returns and factor/ESG data.")
start_date, end_date = common_index_dates.min(), common_index_dates.max()
print(f"Common date range for panel: {start_date.date()} to {end_date.date()}")

monthly_returns_aligned = monthly_returns.loc[common_index_dates]
ff_esg_data_aligned = ff_esg_data.loc[common_index_dates]

# 2. Prepare Dependent Variable (Excess Returns) in Long Format
print("Preparing excess returns in long format...")
excess_returns_long_list = []
rf_aligned = ff_esg_data_aligned["RF"]

for ticker in ev_tickers_returns:
    if ticker in monthly_returns_aligned.columns:
        temp_ret = pd.DataFrame({
            'Return': monthly_returns_aligned[ticker],
            'Ticker': ticker
        }, index=monthly_returns_aligned.index)
        temp_ret = temp_ret.join(rf_aligned.rename('RF'), how='left')
        temp_ret['excess_return'] = temp_ret['Return'] - temp_ret['RF']
        excess_returns_long_list.append(temp_ret[['Ticker', 'excess_return']])

if not excess_returns_long_list:
    raise ValueError("No excess returns could be calculated.")
excess_returns_with_index = pd.concat(excess_returns_long_list)
excess_returns_long = excess_returns_with_index.reset_index()
excess_returns_long.rename(columns={'index': 'Date'}, inplace=True)
excess_returns_long.dropna(subset=['excess_return'], inplace=True)


# 3. Prepare ESG Data in Long Format & Impute
print("Preparing ESG data in long format and imputing missing values...")
esg_data_long_list = []
esg_cols_available = [col for col in ff_esg_data_aligned.columns if '_ESG' in col]

for col in esg_cols_available:
    ticker = col.replace('_ESG', '')
    if ticker not in ev_tickers_esg:
        continue

    temp_esg = ff_esg_data_aligned[[col]].copy()

    if temp_esg[col].isnull().any():
        median_val = temp_esg[col].median()
        if pd.isna(median_val):
            temp_esg[col] = temp_esg[col].fillna(0)
        else:
            temp_esg[col] = temp_esg[col].fillna(median_val)

    temp_esg.rename(columns={col: 'ESG'}, inplace=True)
    temp_esg['Ticker'] = ticker
    temp_esg['Date'] = temp_esg.index
    esg_data_long_list.append(temp_esg)

if not esg_data_long_list:
    print("Warning: No ESG data could be processed into long format.")
    esg_data_long = pd.DataFrame(columns=['Date', 'Ticker', 'ESG'])
else:
    esg_data_long = pd.concat(esg_data_long_list)
    esg_data_long.reset_index(drop=True, inplace=True) # Reset index after concat


# 4. Merge Returns, Factors, and ESG into a single Panel DataFrame
print("Merging returns, factors, and ESG data...")
panel_data = excess_returns_long.copy()
panel_data = pd.merge(panel_data, ff_esg_data_aligned[factor_columns].reset_index().rename(columns={'index':'Date'}), on='Date', how='left')
panel_data = pd.merge(panel_data, esg_data_long[['Date', 'Ticker', 'ESG']], on=['Date', 'Ticker'], how='left')

# 5. Create Lagged ESG Variable
print("Creating lagged ESG variable...")
panel_data.sort_values(by=['Ticker', 'Date'], inplace=True)
panel_data['ESG_lagged'] = panel_data.groupby('Ticker')['ESG'].shift(1)

# 6. Handle Final Missing Values
print("Handling final missing values after merging and lagging...")
essential_cols = ['excess_return', 'ESG_lagged'] + factor_columns[:-1] # Dependent + Lagged ESG + 5 FF factors
initial_rows = len(panel_data)
panel_data.dropna(subset=essential_cols, inplace=True)
rows_after_na = len(panel_data)
print(f"Dropped {initial_rows - rows_after_na} rows due to missing values in essential columns.")

if panel_data.empty:
    raise ValueError("Panel data is empty after merging, lagging, and handling NaNs.")

# 7. Set Panel MultiIndex
print("Setting Panel MultiIndex (Ticker, Date)...")
try:
    panel_data = panel_data.set_index(['Ticker', 'Date'])
except KeyError:
     raise KeyError("Columns 'Ticker' or 'Date' not found before setting index.")


print(f"\nFinal Panel Data Ready. Shape: {panel_data.shape}")
# print("Panel Data Head:\n", panel_data.head())

# --- Panel Regression Analysis ---
print("\nStarting Panel Data Regression (Two-Way Fixed Effects)...")
panel_regression_summary = None

try:
    dependent = panel_data['excess_return']
    exog_vars = ['ESG_lagged']

    if not all(v in panel_data.columns for v in exog_vars):
        missing_vars = [v for v in exog_vars if v not in panel_data.columns]
        raise ValueError(f"Exogenous variable(s) {missing_vars} not found.")
    exog_df_final = panel_data[exog_vars]
    if exog_df_final.empty:
         raise ValueError("No data remaining for exogenous variables.")
    if exog_df_final.var(ddof=0).iloc[0] == 0:
         raise ValueError("ESG_lagged has no variance after processing.")

    exog = exog_df_final

    n_obs = exog.shape[0]
    n_entity = panel_data.index.get_level_values(0).nunique()
    n_time = panel_data.index.get_level_values(1).nunique()
    n_exog = exog.shape[1]
    df_resid = n_obs - n_entity - n_time - n_exog + 1
    if df_resid <= 0:
         raise ValueError(f"Insufficient observations ({n_obs}) for degrees of freedom ({df_resid}).")
    elif n_obs < n_exog + n_entity + n_time:
         print(f"Warning: Low observations ({n_obs}) relative to predictors/effects.")

    # --- Define and Fit the PanelOLS Model ---
    print("Defining PanelOLS model with Entity and Time Fixed Effects...")
    # *** THE FIX IS HERE: Removed check_absorbed=True ***
    mod = PanelOLS(dependent, exog, entity_effects=True, time_effects=True)

    print("Fitting model with clustered standard errors (Entity and Time)...")
    fe_res = mod.fit(cov_type='clustered', cluster_entity=True, cluster_time=True)

    # --- Print the Results ---
    print("\nPanel Regression Results (Two-Way Fixed Effects, FF5 absorbed by Time Effects):")
    print(fe_res)

    # Store results summary object
    panel_regression_summary = fe_res.summary

except ValueError as ve:
     print(f"\nMODEL ESTIMATION ERROR: {ve}")
     panel_regression_summary = f"Error: {ve}"
except Exception as e:
    print(f"\nUNEXPECTED ERROR during panel regression: {e}")
    panel_regression_summary = f"Error: {e}"


# --- Save Results to Excel ---
output_filename = 'panel_regression_results_ff5_lagged_esg_FE.xlsx'
print(f"\nSaving panel regression results to '{output_filename}'...")

if isinstance(panel_regression_summary, str):
    try:
        pd.DataFrame({'Error': [panel_regression_summary]}).to_excel(output_filename, sheet_name='Error', index=False)
        print("Error message saved to Excel.")
    except Exception as save_err:
        print(f"Could not save error message to Excel: {save_err}")

elif panel_regression_summary is not None and hasattr(panel_regression_summary, 'tables') and len(panel_regression_summary.tables) >= 3:
    try:
        with pd.ExcelWriter(output_filename, engine='xlsxwriter') as writer:
            df_top = pd.read_html(StringIO(panel_regression_summary.tables[0].as_html()), header=None, index_col=0)[0]
            df_coeffs = pd.read_html(StringIO(panel_regression_summary.tables[1].as_html()), header=0, index_col=0)[0]
            df_bottom = pd.read_html(StringIO(panel_regression_summary.tables[2].as_html()), header=None, index_col=0)[0]

            df_top.to_excel(writer, sheet_name='Model Summary', header=False)
            df_coeffs.reset_index().to_excel(writer, sheet_name='Coefficients', index=False)
            df_bottom.to_excel(writer, sheet_name='Model Diagnostics', header=False)
        print(f"Panel results successfully saved to '{output_filename}'.")
    except IndexError:
         print("Error: Could not access expected tables (0, 1, 2) from the summary object.")
         try:
             pd.DataFrame({'Error': ["Could not parse summary tables correctly."]}).to_excel(output_filename, sheet_name='Error', index=False)
         except Exception as save_err:
             print(f"Could not save error message to Excel: {save_err}")
    except Exception as e:
        print(f"Error saving panel results to Excel: {e}")
        try:
             pd.DataFrame({'Error': [f"Error during Excel save: {e}"]}).to_excel(output_filename, sheet_name='Error', index=False)
        except Exception as save_err:
             print(f"Could not save error message to Excel: {save_err}")

elif panel_regression_summary is None:
     print("Skipping saving results because panel regression did not produce a summary object.")
else:
    print("Could not save panel results - summary object format unexpected or missing tables.")
    try:
         pd.DataFrame({'Error': ["Summary object format unexpected or missing tables."]}).to_excel(output_filename, sheet_name='Error', index=False)
    except Exception as save_err:
         print(f"Could not save error message to Excel: {save_err}")

Successfully downloaded data for TSLA.
Successfully downloaded data for 1211.HK.
Successfully downloaded data for VOW3.DE.
Successfully downloaded data for NIO.
Successfully downloaded data for RIVN.
Successfully downloaded data for LCID.
Successfully downloaded data for XPEV.
Successfully downloaded data for LI.
Successfully downloaded data for GM.
Successfully downloaded data for F.
Successfully downloaded data for 005380.KS.
Successfully downloaded data for PSNY.
Successfully downloaded data for BMW.DE.

Calculating monthly returns...
Monthly returns calculated.

Loading Fama-French 5-Factor and ESG data from '/content/gd_Developed_5_Factors.csv'...
Data loaded successfully.
Using columns from CSV: ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF', 'TSLA_ESG', '1211.HK_ESG', 'VOW3.DE_ESG', 'NIO_ESG', 'RIVN_ESG', 'LCID_ESG', 'XPEV_ESG', 'LI_ESG', 'GM_ESG', 'F_ESG', 'BMW.DE_ESG', 'PSNY_ESG']

Aligning returns and factor data by date...
Common date range for panel: 2022-01-31 to 2024-12-31
P

### Step-by-Step Implementation
#### Import Required Libraries
#### Fetch Data
#### Calculate Monthly Returns
#### Prepare the Dataset
#### Implement Multiple Linear Regression Without Addressing Multicollinearity
#### Evaluate and Display Results
#### Address Multicollinearity
#### Implement Multiple Linear Regression After Addressing Multicollinearity
#### Evaluate and Display Results Again

In [42]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.style as style
import yfinance as yf
import statsmodels.api as sm
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
import xlsxwriter
import warnings # Import warnings module
from io import StringIO # Import StringIO

# Suppress the specific FutureWarning about inplace modification on a copy
warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
    message="A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.*"
)
# Optional: Suppress the read_html warning if desired
warnings.filterwarnings("ignore", category=FutureWarning, message="Passing literal html to 'read_html' is deprecated*")
# Optional: Suppress RuntimeWarning about divide by zero in statsmodels if needed
# warnings.filterwarnings("ignore", category=RuntimeWarning, message="divide by zero encountered in scalar divide")


# Set the plotting style (optional, kept from original)
style.use("default")
params = {
    "axes.labelsize": 8, "font.size": 8, "legend.fontsize": 8,
    "xtick.labelsize": 8, "ytick.labelsize": 8, "text.usetex": False,
    "font.family": "sans-serif", "axes.spines.top": False, "axes.spines.right": False,
    "grid.color": "grey", "axes.grid": True, "grid.alpha": 0.5, "grid.linestyle": ":",
}
plt.rcParams.update(params)

# List of EV tickers (ensure these match columns in the returns data)
ev_tickers = ["TSLA", "1211.HK", "VOW3.DE", "NIO", "RIVN",
              "LCID", "XPEV", "LI", "GM", "F", "005380.KS", "PSNY", "BMW.DE"]

# --- Data Downloading ---
adj_close_data = pd.DataFrame()
print("Downloading historical stock data...")
for ticker in ev_tickers:
    try:
        # Adjust start date slightly if needed for alignment after resampling
        data = yf.download(ticker, start="2021-12-01", end="2025-03-27", progress=False)

        if data.empty:
            print(f"No data downloaded for {ticker}. Skipping.")
            continue

        adj_close_data[ticker] = data.get("Adj Close", data["Close"])
        print(f"Successfully downloaded data for {ticker}.")

    except Exception as e:
        print(f"Error downloading data for {ticker}: {e}")

# --- Data Processing ---
print("\nCalculating monthly returns...")
if adj_close_data.empty:
    raise ValueError("No stock data was successfully downloaded. Cannot proceed.")

# Use 'ME' for month-end frequency to avoid FutureWarning
monthly_returns = adj_close_data.resample("ME").last().pct_change() * 100
monthly_returns.dropna(how='all', inplace=True)
print("Monthly returns calculated.")

# --- Load the NEW Fama-French 5-Factor and ESG data ---
print("\nLoading Fama-French 5-Factor and ESG data...")
# *** CHANGE 1: Update CSV file path ***
csv_file_path = "/content/gd_Developed_5_Factors.csv"
try:
    _ff_esg_data_raw = pd.read_csv(
        csv_file_path,
        index_col=0, # Assuming 'Date' is the first column
        header=0,
        on_bad_lines='warn'
    )
    print(f"Data loaded from '{csv_file_path}'.")
except FileNotFoundError:
    raise FileNotFoundError(f"Error: The CSV file '{csv_file_path}' was not found.")
except Exception as e:
    raise Exception(f"Error loading CSV file: {e}")

# *** CHANGE 2: Update expected columns based on the NEW dataset header ***
# Make sure these names EXACTLY match the columns in gd_Developed_5_Factors.csv
expected_columns = [
    'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF', # FF5 Factors + RF
    'TSLA_ESG', '1211.HK_ESG', 'VOW3.DE_ESG', 'NIO_ESG', 'RIVN_ESG',
    'LCID_ESG', 'XPEV_ESG', 'LI_ESG', 'GM_ESG', 'F_ESG',
    'BMW.DE_ESG', 'PSNY_ESG'
    # Note: 005380.KS_ESG is excluded as it wasn't in the sample header provided
]

# Filter the loaded data to only include expected columns that actually exist
actual_columns_in_csv = [col for col in expected_columns if col in _ff_esg_data_raw.columns]
if len(actual_columns_in_csv) < 6: # Check if at least the factors + RF are present
     raise ValueError(f"Core Fama-French factors missing in the loaded CSV. Found only: {actual_columns_in_csv}")
ff_esg_data = _ff_esg_data_raw[actual_columns_in_csv].copy() # Use .copy()
print(f"Using columns: {list(ff_esg_data.columns)}")

# --- Manual Data Addition (Optional - Keep if needed for specific columns) ---
# Example: Keep the 1211.HK check/override if necessary for the new dataset too
if '1211.HK_ESG' in ff_esg_data.columns and ff_esg_data['1211.HK_ESG'].isnull().all():
    print("Manually adding/overwriting '1211.HK_ESG' data (check if still needed)...")
    # Using the values provided previously, adjust if dataset is different
    esg_values_1211_HK = [24.55] * 12 + [24.05] * 5 + [23.89] * 5 + [26.69] * 6 + [26.1] * 2 + [26.1] * (len(ff_esg_data) - 30) # Adjust length calculation if needed

    if len(esg_values_1211_HK) != len(ff_esg_data):
         temp_series = pd.Series(esg_values_1211_HK).reindex(range(len(ff_esg_data))).ffill().bfill()
         ff_esg_data['1211.HK_ESG'] = pd.Series(temp_series.values, index=ff_esg_data.index)
         print(f"Adjusted length of manual ESG data for 1211.HK to {len(ff_esg_data)}.")
    else:
        ff_esg_data['1211.HK_ESG'] = pd.Series(esg_values_1211_HK, index=ff_esg_data.index)
    print("'1211.HK_ESG' data potentially added/updated.")


# Convert index to datetime (assuming format YYYYMM)
ff_esg_data.index = pd.to_datetime(ff_esg_data.index.astype(str), format="%Y%m", errors='coerce')
ff_esg_data.index = ff_esg_data.index + pd.offsets.MonthEnd(0)
ff_esg_data.dropna(axis=0, how='all', inplace=True) # Drop rows if index conversion failed

# --- Align DataFrames ---
print("\nAligning dataframes by date...")
if not isinstance(monthly_returns.index, pd.DatetimeIndex):
     monthly_returns.index = pd.to_datetime(monthly_returns.index)
if not isinstance(ff_esg_data.index, pd.DatetimeIndex):
     ff_esg_data.index = pd.to_datetime(ff_esg_data.index)

# Robust date range finding
common_index = monthly_returns.index.intersection(ff_esg_data.index)
if common_index.empty:
    raise ValueError("No overlapping dates found between stock returns and factor data.")
start_date = common_index.min()
end_date = common_index.max()
print(f"Common date range: {start_date.date()} to {end_date.date()}")

# Filter using the common index
monthly_returns_aligned = monthly_returns.loc[common_index]
ff_esg_data_aligned = ff_esg_data.loc[common_index]

# Calculate excess returns
print("\nCalculating excess returns...")
excess_returns = pd.DataFrame(index=monthly_returns_aligned.index)
if "RF" not in ff_esg_data_aligned.columns:
    raise KeyError("The 'RF' column is missing from the Fama-French data after filtering.")
rf_aligned = ff_esg_data_aligned["RF"]

for ticker in ev_tickers:
    if ticker in monthly_returns_aligned.columns:
        # Subtraction aligns automatically on index
        excess_returns[ticker] = monthly_returns_aligned[ticker] - rf_aligned
excess_returns.dropna(how='all', inplace=True)
print("Excess returns calculated.")

# Combine excess returns with FF+ESG data
# Use inner join to ensure only dates present in both remain
final_data = excess_returns.join(ff_esg_data_aligned, how='inner')
print(f"Shape of final_data after join: {final_data.shape}")
if final_data.empty:
     raise ValueError("Final data is empty after joining returns and factors. Check date alignment and data.")

# --- Imputation for ESG columns ONLY ---
print("\nPerforming median imputation for ESG columns...")
esg_cols_in_final = [col for col in final_data.columns if '_ESG' in col]
if not esg_cols_in_final:
    print("Warning: No columns containing '_ESG' found in final_data for imputation.")
else:
    print(f"Found ESG columns for imputation: {esg_cols_in_final}")

for col in esg_cols_in_final:
    if final_data[col].isnull().any():
        median_val = final_data[col].median()
        if pd.isna(median_val):
            print(f"Warning: Median for {col} is NaN. Filling missing values with 0 instead.")
            final_data.loc[:, col] = final_data[col].fillna(0)
        else:
            final_data.loc[:, col] = final_data[col].fillna(median_val)
            print(f"Imputed missing values in {col} with median {median_val:.2f}")
print("Imputation complete.")

# --- Regression Analysis ---
print("\nStarting regression analysis (Fama-French 5-Factor + Aggregate ESG model)...")
regression_results = {}

# Define VIF function
def calculate_vif(X_df):
    vif_data = pd.DataFrame(columns=['feature', 'VIF'])
    if X_df.shape[1] < 2:
        return vif_data
    try:
        vif_data["feature"] = X_df.columns
        vifs = [variance_inflation_factor(X_df.values, i) for i in range(X_df.shape[1])]
        vif_data["VIF"] = vifs
        vif_data['VIF'] = vif_data['VIF'].replace([np.inf, -np.inf], np.nan)
    except Exception as e:
        print(f"Error calculating VIF for {X_df.columns}: {e}")
        return pd.DataFrame({'feature': X_df.columns, 'VIF': [np.nan]*X_df.shape[1]})
    return vif_data


for ticker in ev_tickers:
    print(f"\n--- Processing {ticker} ---")

    if ticker not in final_data.columns:
        print(f"Ticker {ticker} return data not found. Skipping.")
        continue

    y_series = final_data[ticker]
    # *** CHANGE 3: Update base predictor columns for FF5 ***
    base_predictor_cols = ["Mkt-RF", "SMB", "HML", "RMW", "CMA"]
    esg_col_name = f"{ticker}_ESG"
    predictor_cols = base_predictor_cols[:]

    if esg_col_name in final_data.columns:
        predictor_cols.append(esg_col_name)
    else:
        print(f"ESG column {esg_col_name} not found. Running without ESG factor.")

    # Ensure all chosen predictors actually exist in final_data
    predictor_cols = [col for col in predictor_cols if col in final_data.columns]
    # Check if *all 5* base factors are present
    if not all(factor in predictor_cols for factor in base_predictor_cols):
         print(f"Warning: One or more base FF-5 factors missing. Skipping {ticker}.")
         continue

    # Align y and X, Handle NaNs
    required_cols = [ticker] + predictor_cols
    # Check existence before selection
    cols_exist = [col for col in required_cols if col in final_data.columns]
    if len(cols_exist) != len(required_cols):
        print(f"Missing required columns for {ticker} in final_data. Skipping.")
        continue

    temp_df = final_data[cols_exist].copy().dropna()

    # Need enough observations for 5 factors + ESG + constant
    min_obs_needed = len(predictor_cols) + 2
    if temp_df.empty or len(temp_df) < min_obs_needed:
        print(f"Not enough valid overlapping data points ({len(temp_df)} found, need >= {min_obs_needed}). Skipping {ticker}.")
        continue

    if ticker not in temp_df.columns: # Double check y
        print(f"Return data column '{ticker}' lost after dropna. Skipping {ticker}.")
        continue
    y_final = temp_df[ticker]

    # Double check predictors
    final_predictor_cols = [col for col in predictor_cols if col in temp_df.columns]
    if len(final_predictor_cols) != len(predictor_cols):
         print(f"Predictor columns lost after dropna for {ticker}. Skipping.")
         continue

    X_final_no_const = temp_df[final_predictor_cols]
    X_final = sm.add_constant(X_final_no_const, has_constant='add')

    print(f"Processing {ticker} with {len(y_final)} observations.")

    # Calculate VIF (but don't print unless debugging)
    vif_data = calculate_vif(X_final)
    # Optional: print VIF if needed during debugging
    # print(f"VIF for {ticker}:\n{vif_data.to_string(index=False)}\n" + "-"*30)


    # Run OLS Regression and PRINT SUMMARY
    try:
        if X_final.shape[1] <= 1: # Should be at least constant + Mkt-RF
             print(f"Not enough predictors remain for {ticker}. Skipping regression.")
             continue

        model = sm.OLS(y_final, X_final).fit()

        # *** PRINT THE REGRESSION SUMMARY HERE ***
        print(f"\nRegression Results for {ticker}:")
        print(model.summary())
        # ****************************************

        # Store results for saving
        regression_results[ticker] = {
            "model_summary": model.summary(),
            "vif": vif_data
        }

    except Exception as e:
        print(f"Error running regression for {ticker}: {e}")
        regression_results[ticker] = {
             "model_summary": f"Error: {e}",
             "vif": vif_data
        }

# --- Save Results ---
# *** CHANGE 4: Update output filename ***
output_filename = 'regression_results_ff5_aggregate_esg.xlsx'
print(f"\nSaving all regression results to '{output_filename}'...")
# (Saving code remains unchanged)
with pd.ExcelWriter(output_filename, engine='xlsxwriter') as writer:
    for ticker, results in regression_results.items():
        vif_df_to_save = results.get("vif", pd.DataFrame({'feature':[], 'VIF':[]}))

        if isinstance(results["model_summary"], str):
             error_df = pd.DataFrame({'Error': [results["model_summary"]]})
             error_df.to_excel(writer, sheet_name=f'{ticker}_Error', index=False)
             vif_df_to_save.to_excel(writer, sheet_name=f'{ticker}_vif', index=False)

        elif hasattr(results["model_summary"], 'tables') and len(results["model_summary"].tables) >= 3:
            try:
                html_table0 = results["model_summary"].tables[0].as_html()
                summary_t0 = pd.read_html(StringIO(html_table0), header=None, index_col=None)[0]
                summary_t0.to_excel(writer, sheet_name=f'{ticker}_Model', index=False, header=False)

                html_table1 = results["model_summary"].tables[1].as_html()
                summary_t1 = pd.read_html(StringIO(html_table1), header=0, index_col=0)[0]
                summary_t1.reset_index().to_excel(writer, sheet_name=f'{ticker}_Coefs', index=False)

                html_table2 = results["model_summary"].tables[2].as_html()
                summary_t2 = pd.read_html(StringIO(html_table2), header=None, index_col=None)[0]
                summary_t2.to_excel(writer, sheet_name=f'{ticker}_Diag', index=False, header=False)

                vif_df_to_save.to_excel(writer, sheet_name=f'{ticker}_vif', index=False)

            except Exception as e:
                 print(f"Error parsing summary tables for {ticker}: {e}. Saving VIF only.")
                 vif_df_to_save.to_excel(writer, sheet_name=f'{ticker}_vif', index=False)
                 error_df = pd.DataFrame({'Error': [f"Error parsing summary: {e}"]})
                 error_df.to_excel(writer, sheet_name=f'{ticker}_Error', index=False)
        else:
            print(f"Unexpected summary structure or insufficient tables for {ticker}. Saving VIF only.")
            vif_df_to_save.to_excel(writer, sheet_name=f'{ticker}_vif', index=False)
            error_df = pd.DataFrame({'Error': [f"Summary object missing tables for {ticker}."]})
            error_df.to_excel(writer, sheet_name=f'{ticker}_Error', index=False)

print(f"All regression results saved to '{output_filename}'.")

Successfully downloaded data for TSLA.
Successfully downloaded data for 1211.HK.
Successfully downloaded data for VOW3.DE.
Successfully downloaded data for NIO.
Successfully downloaded data for RIVN.
Successfully downloaded data for LCID.
Successfully downloaded data for XPEV.
Successfully downloaded data for LI.
Successfully downloaded data for GM.
Successfully downloaded data for F.
Successfully downloaded data for 005380.KS.
Successfully downloaded data for PSNY.
Successfully downloaded data for BMW.DE.

Calculating monthly returns...
Monthly returns calculated.

Loading Fama-French 5-Factor and ESG data...
Data loaded from '/content/gd_Developed_5_Factors.csv'.
Using columns: ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF', 'TSLA_ESG', '1211.HK_ESG', 'VOW3.DE_ESG', 'NIO_ESG', 'RIVN_ESG', 'LCID_ESG', 'XPEV_ESG', 'LI_ESG', 'GM_ESG', 'F_ESG', 'BMW.DE_ESG', 'PSNY_ESG']

Aligning dataframes by date...
Common date range: 2022-01-31 to 2024-12-31

Calculating excess returns...
Excess returns 

In [ ]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.style as style
import yfinance as yf
import statsmodels.api as sm
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
import xlsxwriter

# Set the plotting style
style.use("default")
params = {
    "axes.labelsize": 8, "font.size": 8, "legend.fontsize": 8,
    "xtick.labelsize": 8, "ytick.labelsize": 8, "text.usetex": False,
    "font.family": "sans-serif", "axes.spines.top": False, "axes.spines.right": False,
    "grid.color": "grey", "axes.grid": True, "grid.alpha": 0.5, "grid.linestyle": ":",
}
plt.rcParams.update(params)

# List of EV tickers
ev_tickers = ["TSLA", "1211.HK", "VOW3.DE", "NIO", "RIVN",
              "LCID", "XPEV", "LI", "GM", "F", "005380.KS", "PSNY", "BMW.DE"]

# Create an empty DataFrame to hold adjusted close prices
adj_close_data = pd.DataFrame()

# Download historical data for each EV ticker
for ticker in ev_tickers:
    try:
        data = yf.download(ticker, start="2022-01-01", end="2025-03-27")  # Updated end date

        if data.empty:
            print(f"No data downloaded for {ticker}.")
            continue

        # Store adjusted close prices
        adj_close_data[ticker] = data.get("Adj Close", data["Close"])

    except Exception as e:
        print(f"Error downloading data for {ticker}: {e}")

# Calculate monthly returns
qqq_monthly = adj_close_data.resample("M").last().pct_change() * 100  # Monthly returns in percentage
qqq_monthly.dropna(inplace=True)

# Load the Fama-French factors with ESG data included
ff_factors_monthly = pd.read_csv(
    "/content/2022_2024_F-F_Research_Data_Factors.CSV",
    index_col=0,
    header=0,
    on_bad_lines='skip'
)

# Define the correct column names based on your updated dataset
ff_factors_monthly.columns = [
    'Mkt-RF', 'SMB', 'HML', 'RF',
    'TSLA_ESG', '1211.HK_ESG', 'VOW3.DE_ESG', 'NIO_ESG',
    'RIVN_ESG', 'LCID_ESG', 'XPEV_ESG', 'LI_ESG', 'GM_ESG', 'F_ESG', 'BMW.DE_ESG'
]

# Manually add the missing ESG values for 1211.HK
# Assuming that these values are the same as in your provided data
#esg_values_1211_HK = [24.55] * 12 + [24.05] * 5 + [23.89] * 5 + [26.69] * 6 + [26.1] * 2
#ff_factors_monthly['1211.HK_ESG'] = esg_values_1211_HK

# Manually add the missing ESG values for 1211.HK
# Assuming these are the values you provided
esg_values_1211_HK = [24.55] * 12 + [24.05] * 5 + [23.89] * 5 + [26.69] * 6 + [26.1] * 2

# Create a DataFrame with the correct length
if len(esg_values_1211_HK) < len(ff_factors_monthly):
    # Fill with the last known value or NaN
    esg_values_1211_HK += [esg_values_1211_HK[-1]] * (len(ff_factors_monthly) - len(esg_values_1211_HK))
elif len(esg_values_1211_HK) > len(ff_factors_monthly):
    # Trim the values if they exceed the length of the DataFrame
    esg_values_1211_HK = esg_values_1211_HK[:len(ff_factors_monthly)]

# Assign the ESG values to the DataFrame
ff_factors_monthly['1211.HK_ESG'] = esg_values_1211_HK

# Convert index to datetime
# ff_factors_monthly.index = pd.to_datetime(ff_factors_monthly.index.astype(str), format="%Y%m", errors='coerce')
# ff_factors_monthly.index = ff_factors_monthly.index + pd.offsets.MonthEnd(0)


# Convert index to datetime
ff_factors_monthly.index = pd.to_datetime(ff_factors_monthly.index.astype(str), format="%Y%m", errors='coerce')
ff_factors_monthly.index = ff_factors_monthly.index + pd.offsets.MonthEnd(0)

# Filter Fama-French factors to match the date range of the stock returns
ff_factors_monthly = ff_factors_monthly[(ff_factors_monthly.index >= '2022-01-01') & (ff_factors_monthly.index <= '2025-03-27')]  # Updated end date

# Ensure that both DataFrames cover the same date range
start_date = max(qqq_monthly.index.min(), ff_factors_monthly.index.min())
end_date = min(qqq_monthly.index.max(), ff_factors_monthly.index.max())

# Filter both DataFrames to the common date range
qqq_monthly = qqq_monthly[(qqq_monthly.index >= start_date) & (qqq_monthly.index <= end_date)]
ff_factors_monthly = ff_factors_monthly[(ff_factors_monthly.index >= start_date) & (ff_factors_monthly.index <= end_date)]

# Calculate excess returns for each EV ticker
excess_returns = pd.DataFrame(index=qqq_monthly.index)

for ticker in ev_tickers:
    if ticker in qqq_monthly.columns:
        excess_returns[ticker] = qqq_monthly[ticker] - ff_factors_monthly["RF"]

# Combine excess returns with ESG data
final_data = excess_returns.join(ff_factors_monthly, how='outer')

# Step 1: Imputation for Missing Values
# Perform median imputation for ESG columns
for col in final_data.columns:
    if 'ESG' in col:
        final_data[col].fillna(final_data[col].median(), inplace=True)

# Step 2: Identify and Remove Highly Collinear ESG Variables
def calculate_vif(X):
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_data

# Step 3: Run the Regression Model for each ticker with ESG variables
regression_results = {}

for ticker in ev_tickers:
    if ticker in excess_returns.columns:
        # Prepare independent variables: Fama-French factors and ESG scores
        X = sm.add_constant(ff_factors_monthly[["Mkt-RF", "SMB", "HML",
            f"{ticker}_ESG"]])  # Only include ESG

        # Ensure y is not empty
        y = excess_returns[ticker].dropna()
        if len(y) == 0:
            print(f"No data for regression on {ticker}.")
            continue

        # Ensure X and y have the same length
        X = X.loc[y.index]

        # Drop rows with NaN values in X or y
        X = X.dropna()
        y = y.loc[X.index]

        if len(y) == 0 or X.empty:
            print(f"No valid data for regression on {ticker} after cleaning.")
            continue

        # Calculate VIF before running the regression
        vif_data = calculate_vif(X)
        print(f"VIF for {ticker}:\n{vif_data}\n")

        # Remove variables with high VIF
        high_vif_columns = vif_data[vif_data['VIF'] > 10]['feature'].tolist()
        if high_vif_columns:
            print(f"Removing columns with high VIF for {ticker}: {high_vif_columns}")
            X = X.drop(columns=high_vif_columns, errors='ignore')

        # Check if X is empty after dropping high VIF columns
        if X.empty:
            print(f"No valid predictors left for regression on {ticker} after removing high VIF columns.")
            continue

        # Run the regression model
        model = sm.OLS(y, X).fit()

        # Store results in the dictionary
        regression_results[ticker] = {
            "model_summary": model.summary(),
            "vif_data": vif_data
        }

        print(f"Regression results for {ticker}:\n{model.summary()}\n")

# Save all regression results to an Excel file
with pd.ExcelWriter('regression_results.xlsx', engine='xlsxwriter') as writer:
    for ticker, results in regression_results.items():
        # Save regression summary as a DataFrame
        summary_df = pd.DataFrame(results["model_summary"].tables[1].data[1:], columns=results["model_summary"].tables[1].data[0])
        summary_df.to_excel(writer, sheet_name=f'{ticker}_summary', index=False)

        # Save VIF as a DataFrame
        results["vif_data"].to_excel(writer, sheet_name=f'{ticker}_vif', index=False)

print("All regression results saved to 'regression_results.xlsx'.")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
<ipython-input-30-d77bf5d393ad>:44: FutureWarning: 'M' is deprecated and will be removed in a future ve

VIF for TSLA:
    feature         VIF
0     const  262.272688
1    Mkt-RF    1.113946
2       SMB    1.127946
3       HML    1.037030
4  TSLA_ESG    1.033305

Removing columns with high VIF for TSLA: ['const']
Regression results for TSLA:
                                 OLS Regression Results                                
Dep. Variable:                   TSLA   R-squared (uncentered):                   0.603
Model:                            OLS   Adj. R-squared (uncentered):              0.552
Method:                 Least Squares   F-statistic:                              11.77
Date:                Fri, 28 Mar 2025   Prob (F-statistic):                    6.28e-06
Time:                        15:07:54   Log-Likelihood:                         -136.11
No. Observations:                  35   AIC:                                      280.2
Df Residuals:                      31   BIC:                                      286.4
Df Model:                           4                    

KeyError: "['005380.KS_ESG'] not in index"

### Reliable result

In [34]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.style as style
import yfinance as yf
import statsmodels.api as sm
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
import xlsxwriter
import warnings # Import warnings module
from io import StringIO # Import StringIO

# Suppress the specific FutureWarning about inplace modification on a copy
warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
    message="A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.*"
)
# Optional: Suppress the read_html warning if desired
warnings.filterwarnings("ignore", category=FutureWarning, message="Passing literal html to 'read_html' is deprecated*")
# Optional: Suppress RuntimeWarning about divide by zero in statsmodels if needed
# warnings.filterwarnings("ignore", category=RuntimeWarning, message="divide by zero encountered in scalar divide")


# Set the plotting style (optional, kept from original)
style.use("default")
params = {
    "axes.labelsize": 8, "font.size": 8, "legend.fontsize": 8,
    "xtick.labelsize": 8, "ytick.labelsize": 8, "text.usetex": False,
    "font.family": "sans-serif", "axes.spines.top": False, "axes.spines.right": False,
    "grid.color": "grey", "axes.grid": True, "grid.alpha": 0.5, "grid.linestyle": ":",
}
plt.rcParams.update(params)

# List of EV tickers
ev_tickers = ["TSLA", "1211.HK", "VOW3.DE", "NIO", "RIVN",
              "LCID", "XPEV", "LI", "GM", "F", "005380.KS", "PSNY", "BMW.DE"]

# --- Data Downloading ---
adj_close_data = pd.DataFrame()
print("Downloading historical stock data...")
for ticker in ev_tickers:
    try:
        data = yf.download(ticker, start="2021-12-01", end="2025-03-27", progress=False)

        if data.empty:
            print(f"No data downloaded for {ticker}. Skipping.")
            continue

        adj_close_data[ticker] = data.get("Adj Close", data["Close"])
        print(f"Successfully downloaded data for {ticker}.")

    except Exception as e:
        print(f"Error downloading data for {ticker}: {e}")

# --- Data Processing ---
print("\nCalculating monthly returns...")
if adj_close_data.empty:
    raise ValueError("No stock data was successfully downloaded. Cannot proceed.")

monthly_returns = adj_close_data.resample("ME").last().pct_change() * 100
monthly_returns.dropna(how='all', inplace=True)
print("Monthly returns calculated.")

# Load the Fama-French factors with ESG data
print("\nLoading Fama-French and ESG data...")
try:
    _ff_esg_data_raw = pd.read_csv(
        "/content/2022_2024_F-F_Research_Data_Factors.CSV",
        index_col=0,
        header=0,
        on_bad_lines='warn'
    )
    print("Fama-French and ESG data loaded.")
except FileNotFoundError:
    raise FileNotFoundError("Error: The CSV file '/content/2022_2024_F-F_Research_Data_Factors.CSV' was not found.")
except Exception as e:
    raise Exception(f"Error loading CSV file: {e}")

# Define expected columns for the AGGREGATE model
expected_columns = [
    'Mkt-RF', 'SMB', 'HML', 'RF',
    'TSLA_ESG', '1211.HK_ESG', 'VOW3.DE_ESG', 'NIO_ESG',
    'RIVN_ESG', 'LCID_ESG', 'XPEV_ESG', 'LI_ESG', 'GM_ESG', 'F_ESG',
    '005380.KS_ESG', 'PSNY_ESG', 'BMW.DE_ESG'
]

actual_columns = [col for col in expected_columns if col in _ff_esg_data_raw.columns]
ff_esg_data = _ff_esg_data_raw[actual_columns].copy()
print(f"Using columns: {list(ff_esg_data.columns)}")

# --- Manually Add 1211.HK_ESG if necessary ---
if '1211.HK_ESG' not in ff_esg_data.columns or ff_esg_data['1211.HK_ESG'].isnull().all():
    print("Manually adding/overwriting '1211.HK_ESG' data...")
    esg_values_1211_HK = [24.55] * 12 + [24.05] * 5 + [23.89] * 5 + [26.69] * 6 + [26.1] * 2

    if len(esg_values_1211_HK) != len(ff_esg_data):
         temp_series = pd.Series(esg_values_1211_HK).reindex(range(len(ff_esg_data))).ffill().bfill()
         ff_esg_data['1211.HK_ESG'] = pd.Series(temp_series.values, index=ff_esg_data.index)
         print(f"Adjusted length of manual ESG data for 1211.HK to {len(ff_esg_data)}.")
    else:
        ff_esg_data['1211.HK_ESG'] = pd.Series(esg_values_1211_HK, index=ff_esg_data.index)
    print("'1211.HK_ESG' data added/updated.")

# Convert index to datetime
ff_esg_data.index = pd.to_datetime(ff_esg_data.index.astype(str), format="%Y%m", errors='coerce')
ff_esg_data.index = ff_esg_data.index + pd.offsets.MonthEnd(0)
ff_esg_data.dropna(axis=0, how='all', inplace=True)

# --- Align DataFrames ---
print("\nAligning dataframes by date...")
if not isinstance(monthly_returns.index, pd.DatetimeIndex):
     monthly_returns.index = pd.to_datetime(monthly_returns.index)
if not isinstance(ff_esg_data.index, pd.DatetimeIndex):
     ff_esg_data.index = pd.to_datetime(ff_esg_data.index)

start_date = max(monthly_returns.index.min(), ff_esg_data.index.min())
end_date = min(monthly_returns.index.max(), ff_esg_data.index.max())
print(f"Common date range: {start_date.date()} to {end_date.date()}")

monthly_returns_aligned = monthly_returns[(monthly_returns.index >= start_date) & (monthly_returns.index <= end_date)]
ff_esg_data_aligned = ff_esg_data[(ff_esg_data.index >= start_date) & (ff_esg_data.index <= end_date)]

# Calculate excess returns
print("\nCalculating excess returns...")
excess_returns = pd.DataFrame(index=monthly_returns_aligned.index)
if "RF" not in ff_esg_data_aligned.columns:
    raise KeyError("The 'RF' column is missing from the Fama-French data after filtering.")
rf_aligned = ff_esg_data_aligned["RF"]

for ticker in ev_tickers:
    if ticker in monthly_returns_aligned.columns:
        stock_returns_aligned = monthly_returns_aligned[ticker].reindex(rf_aligned.index)
        excess_returns[ticker] = stock_returns_aligned - rf_aligned
excess_returns.dropna(how='all', inplace=True)
print("Excess returns calculated.")

# Combine excess returns with FF+ESG data (using aligned data)
final_data = excess_returns.join(ff_esg_data_aligned, how='inner')
print(f"Shape of final_data after join: {final_data.shape}")
if final_data.empty:
     raise ValueError("Final data is empty after joining returns and factors. Check date alignment and data.")

# --- Imputation for ESG columns ONLY ---
print("\nPerforming median imputation for ESG columns...")
esg_cols_in_final = [col for col in final_data.columns if '_ESG' in col]
if not esg_cols_in_final:
    print("Warning: No columns containing '_ESG' found in final_data for imputation.")
else:
    print(f"Found ESG columns for imputation: {esg_cols_in_final}")

for col in esg_cols_in_final:
    if final_data[col].isnull().any():
        median_val = final_data[col].median()
        if pd.isna(median_val):
            print(f"Warning: Median for {col} is NaN. Filling missing values with 0 instead.")
            final_data.loc[:, col] = final_data[col].fillna(0)
        else:
            final_data.loc[:, col] = final_data[col].fillna(median_val)
            print(f"Imputed missing values in {col} with median {median_val:.2f}")
print("Imputation complete.")

# --- Regression Analysis ---
print("\nStarting regression analysis (Aggregate ESG model)...")
regression_results = {}

# Define VIF function
def calculate_vif(X_df):
    vif_data = pd.DataFrame(columns=['feature', 'VIF'])
    if X_df.shape[1] < 2:
        # print("Warning: VIF calculation requires at least two columns. Skipping VIF.") # Keep this silent for now
        return vif_data
    try:
        vif_data["feature"] = X_df.columns
        vifs = [variance_inflation_factor(X_df.values, i) for i in range(X_df.shape[1])]
        vif_data["VIF"] = vifs
        vif_data['VIF'] = vif_data['VIF'].replace([np.inf, -np.inf], np.nan)
    except Exception as e:
        print(f"Error calculating VIF for {X_df.columns}: {e}")
        return pd.DataFrame({'feature': X_df.columns, 'VIF': [np.nan]*X_df.shape[1]})
    return vif_data


for ticker in ev_tickers:
    print(f"\n--- Processing {ticker} ---")

    if ticker not in final_data.columns:
        print(f"Ticker {ticker} return data not found. Skipping.")
        continue

    y_series = final_data[ticker]
    base_predictor_cols = ["Mkt-RF", "SMB", "HML"]
    esg_col_name = f"{ticker}_ESG"
    predictor_cols = base_predictor_cols[:]

    if esg_col_name in final_data.columns:
        # print(f"Found ESG column: {esg_col_name}") # Keep silent
        predictor_cols.append(esg_col_name)
    else:
        print(f"ESG column {esg_col_name} not found. Running without ESG factor.")

    predictor_cols = [col for col in predictor_cols if col in final_data.columns]
    if len(predictor_cols) < len(base_predictor_cols):
         print(f"Warning: Base FF factors missing. Skipping {ticker}.")
         continue

    required_cols = [ticker] + predictor_cols
    cols_exist = [col for col in required_cols if col in final_data.columns]
    if len(cols_exist) != len(required_cols):
        print(f"Missing required columns for {ticker}. Skipping.")
        continue

    temp_df = final_data[cols_exist].copy().dropna()

    min_obs_needed = len(predictor_cols) + 2
    if temp_df.empty or len(temp_df) < min_obs_needed:
        print(f"Not enough valid overlapping data points ({len(temp_df)} found, need >= {min_obs_needed}). Skipping {ticker}.")
        continue

    if ticker not in temp_df.columns:
        print(f"Return data column '{ticker}' lost after dropna. Skipping {ticker}.")
        continue
    y_final = temp_df[ticker]

    final_predictor_cols = [col for col in predictor_cols if col in temp_df.columns]
    if len(final_predictor_cols) != len(predictor_cols):
         print(f"Predictor columns lost after dropna for {ticker}. Skipping.")
         continue

    X_final_no_const = temp_df[final_predictor_cols]
    X_final = sm.add_constant(X_final_no_const, has_constant='add')

    print(f"Processing {ticker} with {len(y_final)} observations.")

    # Calculate VIF (but don't print it here)
    vif_data = calculate_vif(X_final)

    # Check VIF (excluding const) - Optional warning
    # high_vif_threshold = 10
    # if not vif_data.empty:
    #     vif_check = vif_data[vif_data['feature'] != 'const']
    #     high_vif_actual_predictors = vif_check[vif_check['VIF'] > high_vif_threshold]['feature'].tolist()
    #     if high_vif_actual_predictors:
    #         print(f"WARNING: High VIF detected for non-constant predictors.")


    # Run OLS Regression and PRINT SUMMARY
    try:
        if X_final.shape[1] <= 1:
             print(f"Not enough predictors remain for {ticker}. Skipping regression.")
             continue

        model = sm.OLS(y_final, X_final).fit()

        # *** PRINT THE REGRESSION SUMMARY HERE ***
        print(f"\nRegression Results for {ticker}:")
        print(model.summary())
        # ****************************************

        # Store results for saving
        regression_results[ticker] = {
            "model_summary": model.summary(),
            "vif": vif_data
        }
        # print(f"Successfully ran regression for {ticker}.") # Can remove this if summary is printed

    except Exception as e:
        print(f"Error running regression for {ticker}: {e}")
        regression_results[ticker] = {
             "model_summary": f"Error: {e}",
             "vif": vif_data
        }

# --- Save Results (Keep this section as is) ---
output_filename = 'regression_results_aggregate_esg.xlsx'
print(f"\nSaving all regression results to '{output_filename}'...")
# (Saving code remains unchanged from the previous version)
with pd.ExcelWriter(output_filename, engine='xlsxwriter') as writer:
    for ticker, results in regression_results.items():
        vif_df_to_save = results.get("vif", pd.DataFrame({'feature':[], 'VIF':[]}))

        if isinstance(results["model_summary"], str):
             error_df = pd.DataFrame({'Error': [results["model_summary"]]})
             error_df.to_excel(writer, sheet_name=f'{ticker}_Error', index=False)
             vif_df_to_save.to_excel(writer, sheet_name=f'{ticker}_vif', index=False)

        elif hasattr(results["model_summary"], 'tables') and len(results["model_summary"].tables) >= 3:
            try:
                html_table0 = results["model_summary"].tables[0].as_html()
                summary_t0 = pd.read_html(StringIO(html_table0), header=None, index_col=None)[0]
                summary_t0.to_excel(writer, sheet_name=f'{ticker}_Model', index=False, header=False)

                html_table1 = results["model_summary"].tables[1].as_html()
                summary_t1 = pd.read_html(StringIO(html_table1), header=0, index_col=0)[0]
                summary_t1.reset_index().to_excel(writer, sheet_name=f'{ticker}_Coefs', index=False)

                html_table2 = results["model_summary"].tables[2].as_html()
                summary_t2 = pd.read_html(StringIO(html_table2), header=None, index_col=None)[0]
                summary_t2.to_excel(writer, sheet_name=f'{ticker}_Diag', index=False, header=False)

                vif_df_to_save.to_excel(writer, sheet_name=f'{ticker}_vif', index=False)

            except Exception as e:
                 print(f"Error parsing summary tables for {ticker}: {e}. Saving VIF only.")
                 vif_df_to_save.to_excel(writer, sheet_name=f'{ticker}_vif', index=False)
                 error_df = pd.DataFrame({'Error': [f"Error parsing summary: {e}"]})
                 error_df.to_excel(writer, sheet_name=f'{ticker}_Error', index=False)
        else:
            print(f"Unexpected summary structure or insufficient tables for {ticker}. Saving VIF only.")
            vif_df_to_save.to_excel(writer, sheet_name=f'{ticker}_vif', index=False)
            error_df = pd.DataFrame({'Error': [f"Summary object missing tables for {ticker}."]})
            error_df.to_excel(writer, sheet_name=f'{ticker}_Error', index=False)

print(f"All regression results saved to '{output_filename}'.")

Successfully downloaded data for TSLA.
Successfully downloaded data for 1211.HK.
Successfully downloaded data for VOW3.DE.
Successfully downloaded data for NIO.
Successfully downloaded data for RIVN.
Successfully downloaded data for LCID.
Successfully downloaded data for XPEV.
Successfully downloaded data for LI.
Successfully downloaded data for GM.
Successfully downloaded data for F.
Successfully downloaded data for 005380.KS.
Successfully downloaded data for PSNY.
Successfully downloaded data for BMW.DE.

Calculating monthly returns...
Monthly returns calculated.

Loading Fama-French and ESG data...
Fama-French and ESG data loaded.
Using columns: ['Mkt-RF', 'SMB', 'HML', 'RF', 'TSLA_ESG', '1211.HK_ESG', 'VOW3.DE_ESG', 'NIO_ESG', 'RIVN_ESG', 'LCID_ESG', 'XPEV_ESG', 'LI_ESG', 'GM_ESG', 'F_ESG', 'PSNY_ESG', 'BMW.DE_ESG']

Aligning dataframes by date...
Common date range: 2022-01-31 to 2024-12-31

Calculating excess returns...
Excess returns calculated.
Shape of final_data after join: (3

/usr/local/lib/python3.11/dist-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.11/dist-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss



Regression Results for F:
                            OLS Regression Results                            
Dep. Variable:                      F   R-squared:                       0.627
Model:                            OLS   Adj. R-squared:                  0.579
Method:                 Least Squares   F-statistic:                     13.02
Date:                Fri, 28 Mar 2025   Prob (F-statistic):           2.47e-06
Time:                        15:45:11   Log-Likelihood:                -125.09
No. Observations:                  36   AIC:                             260.2
Df Residuals:                      31   BIC:                             268.1
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -12.9141    

/usr/local/lib/python3.11/dist-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


All regression results saved to 'regression_results_aggregate_esg.xlsx'.
